# Tutoriel DSPy et GEPA : Classification Automatique de Tickets IT

Ce tutoriel complet vous guide de manière progressive à travers **DSPy** (Declarative Self-improving Language Programs) et **GEPA** (Genetic-Pareto Algorithm).

## Objectifs d'apprentissage

1. Maîtriser les **Signatures** : définir les entrées/sorties de vos tâches
2. Comprendre les **Modules** : les différents types et comment les composer
3. Évaluer les performances avec des **métriques**
4. Optimiser automatiquement avec les **Optimiseurs**
5. Utiliser différents **modèles de langage** facilement
6. Maîtriser **GEPA** pour l'optimisation avancée

## Prérequis

- Python 3.9+
- Ollama installé et en cours d'exécution
- Un modèle téléchargé (ex: `ollama pull llama3.1:8b`)

## Structure du tutoriel

1. **Configuration et Données** - Préparer l'environnement
2. **Les Signatures DSPy** - Définir ce que vous voulez faire
3. **Les Modules DSPy** - Comment le faire
4. **L'Évaluation** - Mesurer les performances
5. **Les Optimiseurs** - Améliorer automatiquement
6. **Multi-Modèles** - Flexibilité entre providers
7. **GEPA en Pratique** - Optimisation avancée

## Configuration initiale

Commençons par installer les dépendances et configurer notre environnement.

In [ ]:
# Installation des dépendances (décommentez si nécessaire)
# !uv pip install dspy-ai

In [39]:
import dspy
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques importées avec succès!")

✅ Bibliothèques importées avec succès!


### Configuration du modèle de langage

Nous utilisons Ollama avec Llama 3.1, un modèle open-source qui fonctionne localement.

In [40]:
print("🚀 Configuration de DSPy avec Ollama...")

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

print("✅ DSPy configuré avec Ollama Llama 3.1:8b")

🚀 Configuration de DSPy avec Ollama...
✅ DSPy configuré avec Ollama Llama 3.1:8b


### Préparation des données

Nous travaillons avec des tickets IT en français. Chaque ticket contient :
- Une **description** du problème
- Une **catégorie** (Hardware, Software, Network, etc.)
- Une **priorité** (Low, Medium, High, Urgent, Critical)

In [41]:
# Données d'entraînement
trainset = [
    {"ticket": "Mon ordinateur ne démarre plus depuis ce matin. J'ai une présentation importante dans 2 heures.", "category": "Hardware", "priority": "Urgent"},
    {"ticket": "Je n'arrive pas à me connecter à l'imprimante du 3e étage. Ça peut attendre.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le VPN ne fonctionne plus. Impossible d'accéder aux fichiers du serveur.", "category": "Network", "priority": "High"},
    {"ticket": "J'ai oublié mon mot de passe Outlook. Je peux utiliser le webmail.", "category": "Account", "priority": "Medium"},
    {"ticket": "Le site web affiche une erreur 500. Les clients ne peuvent plus commander!", "category": "Application", "priority": "Critical"},
    {"ticket": "Ma souris sans fil ne répond plus bien. Les piles sont faibles.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le système de paie ne calcule pas les heures supplémentaires. C'est la fin du mois.", "category": "Application", "priority": "Urgent"},
    {"ticket": "J'aimerais une mise à jour de mon logiciel Adobe quand vous aurez le temps.", "category": "Software", "priority": "Low"},
    {"ticket": "Le serveur de base de données est très lent. Toute la production est impactée.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "Je ne reçois plus les emails. J'attends des réponses de fournisseurs.", "category": "Email", "priority": "High"},
    {"ticket": "Mon écran externe ne s'affiche plus. Je peux travailler sur le laptop.", "category": "Hardware", "priority": "Medium"},
    {"ticket": "Le wifi de la salle A ne fonctionne pas. Réunion avec des externes dans 30 min.", "category": "Network", "priority": "Urgent"},
    {"ticket": "Je voudrais installer Slack pour mieux collaborer avec l'équipe.", "category": "Software", "priority": "Medium"},
    {"ticket": "Le système de sauvegarde a échoué cette nuit selon le rapport.", "category": "Infrastructure", "priority": "High"},
    {"ticket": "Mon clavier a une touche qui colle. C'est gérable mais ennuyeux.", "category": "Peripherals", "priority": "Low"}
]

# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

print(f"📊 Données chargées : {len(trainset)} entraînement, {len(valset)} validation")
print(f"📦 {len(CATEGORIES)} catégories, {len(PRIORITIES)} priorités")

📊 Données chargées : 15 entraînement, 7 validation
📦 8 catégories, 5 priorités


# Partie 1 : Les signatures DSPy

## Qu'est-ce qu'une signature ?

Une **signature** dans DSPy est une **déclaration de ce que votre programme doit faire**, pas comment le faire.

### Analogie
Imaginez que vous engagez un assistant :
- ❌ **Sans DSPy** : Vous lui donnez des instructions détaillées ("d'abord lis le ticket, ensuite regarde si c'est hardware...")
- ✅ **Avec DSPy** : Vous lui donnez simplement le contrat ("je te donne un ticket, tu me donnes la catégorie et la priorité")

### Composants d'une signature

1. **Docstring** : Description de la tâche
2. **Champs d'entrée** (`InputField`) : Ce que vous fournissez
3. **Champs de sortie** (`OutputField`) : Ce que vous attendez
4. **Descriptions** (optionnelles) : Précisions sur chaque champ

### Exemple 1 : Signature simple

La forme la plus basique d'une signature.

In [42]:
class BasicSignature(dspy.Signature):
    """Classifier un ticket IT."""
    
    ticket = dspy.InputField()
    category = dspy.OutputField()

print("✅ Signature simple créée")
print("   - 1 entrée : ticket")
print("   - 1 sortie : category")

✅ Signature simple créée
   - 1 entrée : ticket
   - 1 sortie : category


### Exemple 2 : Signature avec descriptions

Ajouter des descriptions aide le modèle à mieux comprendre la tâche.

In [43]:
class DescriptiveSignature(dspy.Signature):
    """Classifier un ticket de support IT selon sa catégorie."""
    
    ticket = dspy.InputField(desc="Description du problème rapporté par l'utilisateur")
    category = dspy.OutputField(desc="Catégorie technique du problème")

print("✅ Signature avec descriptions créée")
print("   Les descriptions aident le modèle à mieux comprendre")

✅ Signature avec descriptions créée
   Les descriptions aident le modèle à mieux comprendre


### Exemple 3 : Signature avec contraintes

Spécifier les valeurs possibles dans la description.

In [44]:
class ConstrainedSignature(dspy.Signature):
    """Classifier un ticket IT selon catégorie et priorité."""
    
    ticket = dspy.InputField(desc="Description du ticket de support IT")
    category = dspy.OutputField(desc=f"Catégorie parmi: {', '.join(CATEGORIES)}")
    priority = dspy.OutputField(desc=f"Priorité parmi: {', '.join(PRIORITIES)}")

print("✅ Signature avec contraintes créée")
print("   Les contraintes guident le modèle vers des réponses valides")

✅ Signature avec contraintes créée
   Les contraintes guident le modèle vers des réponses valides


### Exemple 4 : Signature avec contexte supplémentaire

Ajouter des entrées contextuelles pour des tâches complexes.

In [45]:
class ContextualSignature(dspy.Signature):
    """Classifier un ticket en tenant compte de l'historique utilisateur."""
    
    ticket = dspy.InputField(desc="Description du problème actuel")
    user_history = dspy.InputField(desc="Historique des tickets précédents de l'utilisateur")
    category = dspy.OutputField(desc=f"Catégorie parmi: {', '.join(CATEGORIES)}")
    priority = dspy.OutputField(desc=f"Priorité parmi: {', '.join(PRIORITIES)}")
    reasoning = dspy.OutputField(desc="Explication de la décision")

print("✅ Signature contextuelle créée")
print("   - 2 entrées : ticket + historique")
print("   - 3 sorties : catégorie + priorité + raisonnement")

✅ Signature contextuelle créée
   - 2 entrées : ticket + historique
   - 3 sorties : catégorie + priorité + raisonnement


### Exemple 5 : Signature avec format de sortie structuré

Demander un format particulier pour la sortie.

In [46]:
class StructuredOutputSignature(dspy.Signature):
    """Analyser un ticket et produire un rapport structuré."""
    
    ticket = dspy.InputField(desc="Description du ticket")
    category = dspy.OutputField(desc="Catégorie technique")
    priority = dspy.OutputField(desc="Niveau de priorité")
    estimated_time = dspy.OutputField(desc="Temps estimé de résolution en heures")
    required_skills = dspy.OutputField(desc="Compétences requises (liste séparée par des virgules)")

print("✅ Signature avec sortie structurée créée")
print("   Utile pour générer des rapports complets")

✅ Signature avec sortie structurée créée
   Utile pour générer des rapports complets


### 💡 Bonnes Pratiques pour les signatures

#### ✅ À faire

1. **Docstring claire** : Décrivez la tâche en une phrase
2. **Noms explicites** : `ticket` plutôt que `input`, `category` plutôt que `output`
3. **Descriptions précises** : Ajoutez `desc` pour guider le modèle
4. **Contraintes claires** : Listez les valeurs possibles quand applicable
5. **Commencer simple** : Ajoutez des champs progressivement

#### ❌ À éviter

1. **Trop de champs** : Commencez avec 1-3 sorties maximum
2. **Descriptions vagues** : "texte" → "description du problème utilisateur"
3. **Noms génériques** : `input1`, `output1` → `ticket`, `category`
4. **Instructions dans le nom** : Le nom décrit le contenu, pas l'action

### 🎯 Signature pour notre tutoriel

Nous utiliserons cette signature pour le reste du tutoriel :

In [47]:
class TicketClassifier(dspy.Signature):
    """Classifier un ticket de support IT selon sa catégorie et sa priorité."""
    
    ticket = dspy.InputField(desc="Description du ticket de support IT")
    category = dspy.OutputField(desc=f"Catégorie parmi: {', '.join(CATEGORIES)}")
    priority = dspy.OutputField(desc=f"Priorité parmi: {', '.join(PRIORITIES)}")

print("✅ Signature principale définie : TicketClassifier")
print("   Cette signature sera utilisée dans tout le tutoriel")

✅ Signature principale définie : TicketClassifier
   Cette signature sera utilisée dans tout le tutoriel


---
# Partie 2 : les modules DSPy

## Qu'est-ce qu'un module ?

Un **module** dans DSPy est un composant qui **utilise une signature** pour générer des prédictions.

### Analogie
- **Signature** = Le contrat ("je te donne X, tu me donnes Y")
- **Module** = L'employé qui exécute le contrat (avec sa propre méthode de travail)

### Les différents types de modules

DSPy offre plusieurs modules, chacun avec une stratégie différente :

1. **Predict** : Génération directe (le plus simple)
2. **ChainOfThought** : Raisonnement avant de répondre
3. **ReAct** : Raisonnement avec actions possibles
4. **ProgramOfThought** : Génération de code pour raisonner
5. **Modules personnalisés** : Composition de plusieurs modules

## Module 1 : Predict (le plus simple)

**Predict** est le module de base : il génère directement une réponse.

### Fonctionnement
1. Reçoit les entrées
2. Génère immédiatement les sorties
3. Retourne le résultat

### Quand l'utiliser
- Tâches simples
- Besoin de rapidité
- Première version d'un système

In [48]:
# Créer un module Predict avec notre signature
predict_classifier = dspy.Predict(TicketClassifier)

# Tester sur un exemple
test_ticket = "Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure."
result = predict_classifier(ticket=test_ticket)

print("🔮 Module : Predict")
print(f"📝 Ticket : {test_ticket}")
print(f"📦 Catégorie prédite : {result.category}")
print(f"⚡ Priorité prédite : {result.priority}")

🔮 Module : Predict
📝 Ticket : Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure.
📦 Catégorie prédite : Hardware
⚡ Priorité prédite : Urgent


In [49]:
dspy.inspect_history(n=3)





[2025-10-15T13:50:28.356017]

System message:

Your input fields are:
1. `ticket` (str): Description du ticket de support IT
Your output fields are:
1. `reasoning` (str): 
2. `category` (str): Catégorie parmi: Hardware, Software, Network, Application, Infrastructure, Account, Email, Peripherals
3. `priority` (str): Priorité parmi: Low, Medium, High, Urgent, Critical
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## ticket ## ]]
{ticket}

[[ ## reasoning ## ]]
{reasoning}

[[ ## category ## ]]
{category}

[[ ## priority ## ]]
{priority}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classifier un ticket de support IT selon sa catégorie et sa priorité.


User message:

[[ ## ticket ## ]]
Mon antivirus affiche un message d'expiration mais tout fonctionne.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## category ## ]]`, then `[[ ## priorit

## Module 2 : ChainOfThought (avec raisonnement)

**ChainOfThought** demande au modèle de raisonner avant de répondre.

### Fonctionnement
1. Reçoit les entrées
2. **Génère d'abord un raisonnement**
3. Génère ensuite les sorties basées sur ce raisonnement
4. Retourne le résultat (avec le raisonnement)

### Quand l'utiliser
- Tâches complexes nécessitant de la réflexion
- Besoin d'expliquer les décisions
- Améliorer la précision (+5-15% typiquement)

### Avantages
- ✅ Meilleure précision
- ✅ Raisonnement inspectable
- ✅ Plus robuste sur des cas complexes

In [50]:
# Créer un module ChainOfThought avec notre signature
cot_classifier = dspy.ChainOfThought(TicketClassifier)

# Tester sur le même exemple
result = cot_classifier(ticket=test_ticket)

print("🧠 Module : ChainOfThought")
print(f"📝 Ticket : {test_ticket}")
print(f"💭 Raisonnement : {getattr(result, 'rationale', 'non disponible')}")
print(f"📦 Catégorie prédite : {result.category}")
print(f"⚡ Priorité prédite : {result.priority}")

🧠 Module : ChainOfThought
📝 Ticket : Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure.
💭 Raisonnement : non disponible
📦 Catégorie prédite : Hardware
⚡ Priorité prédite : High


### Comparaison Predict vs ChainOfThought

Testons les deux modules sur plusieurs exemples pour voir la différence.

In [51]:
# Tester sur 3 exemples
test_cases = valset[:3]

print("="*70)
print("Comparaison Predict vs ChainOfThought")
print("="*70 + "\n")

for i, example in enumerate(test_cases, 1):
    print(f"--- Exemple {i} ---")
    print(f"Ticket : {example['ticket']}")
    print(f"Attendu : {example['category']} | {example['priority']}\n")
    
    # Predict
    pred_result = predict_classifier(ticket=example['ticket'])
    print(f"  Predict : {pred_result.category} | {pred_result.priority}")
    
    # ChainOfThought
    cot_result = cot_classifier(ticket=example['ticket'])
    print(f"  ChainOfThought : {cot_result.category} | {cot_result.priority}")
    print()

Comparaison Predict vs ChainOfThought

--- Exemple 1 ---
Ticket : Le serveur de fichiers est inaccessible. Personne ne peut travailler.
Attendu : Infrastructure | Critical

  Predict : Infrastructure | High
  ChainOfThought : Infrastructure | High

--- Exemple 2 ---
Ticket : J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.
Attendu : Account | Urgent

  Predict : Account | Urgent
  ChainOfThought : Account | Urgent

--- Exemple 3 ---
Ticket : L'écran de mon collègue en vacances clignote. On peut attendre.
Attendu : Hardware | Low

  Predict : Hardware | Low
  ChainOfThought : Hardware | Low



## Module 3 : ReAct (raisonnement + actions)

**ReAct** alterne entre raisonnement et actions.

### Fonctionnement
1. Raisonne sur le problème
2. Décide d'une action à faire (ex: chercher dans une base de données)
3. Observe le résultat de l'action
4. Raisonne à nouveau avec cette nouvelle information
5. Répète jusqu'à avoir la réponse

### Quand l'utiliser
- Besoin d'interactions avec des outils externes
- Recherche d'informations nécessaire
- Tâches multi-étapes

**Note** : ReAct nécessite de définir des outils/actions disponibles. C'est plus avancé, nous ne le couvrirons pas en détail ici.

## Module 4 : ProgramOfThought (génération de code)

**ProgramOfThought** génère du code Python pour raisonner.

### Fonctionnement
1. Analyse le problème
2. Génère du code Python pour le résoudre
3. Exécute le code
4. Utilise le résultat pour générer la réponse

### Quand l'utiliser
- Problèmes mathématiques
- Calculs complexes
- Manipulation de données structurées

**Exemple typique** : "Combien font 347 * 892 + 123 / 7 ?"
- Le modèle génère : `result = 347 * 892 + 123 / 7`
- Exécute le code : `309541.57`
- Retourne la réponse

**Note** : Moins pertinent pour notre classification de tickets, mais très utile pour d'autres tâches.

## Module 5 : modules personnalisés (composition)

Vous pouvez créer vos propres modules en **composant** plusieurs modules existants.

### Pourquoi composer des modules ?
- Décomposer une tâche complexe en sous-tâches
- Réutiliser des modules existants
- Créer des pipelines sophistiqués

### Exemple 1 : pipeline séquentiel

Classifier d'abord la catégorie, puis la priorité en fonction de la catégorie.

In [52]:
# Définir des signatures spécialisées
class CategoryClassifier(dspy.Signature):
    """Déterminer la catégorie technique d'un ticket IT."""
    ticket = dspy.InputField(desc="Description du ticket")
    category = dspy.OutputField(desc=f"Catégorie parmi: {', '.join(CATEGORIES)}")

class PriorityClassifier(dspy.Signature):
    """Déterminer la priorité d'un ticket en fonction de sa catégorie."""
    ticket = dspy.InputField(desc="Description du ticket")
    category = dspy.InputField(desc="Catégorie technique déjà identifiée")
    priority = dspy.OutputField(desc=f"Priorité parmi: {', '.join(PRIORITIES)}")

# Créer un module composé
class SequentialClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.category_predictor = dspy.ChainOfThought(CategoryClassifier)
        self.priority_predictor = dspy.ChainOfThought(PriorityClassifier)
    
    def forward(self, ticket):
        # Étape 1 : Prédire la catégorie
        category_result = self.category_predictor(ticket=ticket)
        
        # Étape 2 : Prédire la priorité en utilisant la catégorie
        priority_result = self.priority_predictor(
            ticket=ticket,
            category=category_result.category
        )
        
        # Retourner les deux résultats
        return dspy.Prediction(
            category=category_result.category,
            priority=priority_result.priority
        )

# Tester le module composé
sequential = SequentialClassifier()
result = sequential(ticket=test_ticket)

print("🔗 Module composé : SequentialClassifier")
print(f"📝 Ticket : {test_ticket}")
print(f"📦 Catégorie (étape 1) : {result.category}")
print(f"⚡ Priorité (étape 2, basée sur catégorie) : {result.priority}")

🔗 Module composé : SequentialClassifier
📝 Ticket : Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure.
📦 Catégorie (étape 1) : Hardware
⚡ Priorité (étape 2, basée sur catégorie) : Urgent


### Exemple 2 : module avec validation

Ajouter une étape de validation pour vérifier que les prédictions sont valides.

In [53]:
class ValidatedClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classifier = dspy.ChainOfThought(TicketClassifier)
    
    def forward(self, ticket):
        # Prédiction
        result = self.classifier(ticket=ticket)
        
        # Validation de la catégorie
        if result.category not in CATEGORIES:
            print(f"⚠️ Catégorie invalide '{result.category}', correction...")
            result.category = "Application"  # Valeur par défaut
        
        # Validation de la priorité
        if result.priority not in PRIORITIES:
            print(f"⚠️ Priorité invalide '{result.priority}', correction...")
            result.priority = "Medium"  # Valeur par défaut
        
        return result

# Tester le module avec validation
validated = ValidatedClassifier()
result = validated(ticket=test_ticket)

print("\n✅ Module avec validation : ValidatedClassifier")
print(f"📦 Catégorie validée : {result.category}")
print(f"⚡ Priorité validée : {result.priority}")


✅ Module avec validation : ValidatedClassifier
📦 Catégorie validée : Hardware
⚡ Priorité validée : High


### Exemple 3 : module avec consensus (ensemble)

Utiliser plusieurs modules et combiner leurs prédictions.

In [54]:
from collections import Counter

class EnsembleClassifier(dspy.Module):
    def __init__(self, n_models=3):
        super().__init__()
        # Créer plusieurs modules
        self.classifiers = [
            dspy.ChainOfThought(TicketClassifier)
            for _ in range(n_models)
        ]
    
    def forward(self, ticket):
        # Collecter les prédictions de tous les modèles
        categories = []
        priorities = []
        
        for classifier in self.classifiers:
            result = classifier(ticket=ticket)
            categories.append(result.category)
            priorities.append(result.priority)
        
        # Vote majoritaire
        category_vote = Counter(categories).most_common(1)[0][0]
        priority_vote = Counter(priorities).most_common(1)[0][0]
        
        return dspy.Prediction(
            category=category_vote,
            priority=priority_vote
        )

# Tester l'ensemble
print("🗳️ Module ensemble : EnsembleClassifier")
print("   (Combine 3 prédictions par vote majoritaire)")
print(f"\n📝 Ticket : {test_ticket}")

ensemble = EnsembleClassifier(n_models=3)
result = ensemble(ticket=test_ticket)

print(f"📦 Catégorie (consensus) : {result.category}")
print(f"⚡ Priorité (consensus) : {result.priority}")

🗳️ Module ensemble : EnsembleClassifier
   (Combine 3 prédictions par vote majoritaire)

📝 Ticket : Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure.
📦 Catégorie (consensus) : Hardware
⚡ Priorité (consensus) : High


## 💡 Bonnes pratiques pour les modules

### ✅ À faire

1. **Commencer simple** : Utilisez d'abord `Predict`, puis `ChainOfThought` si besoin
2. **Nommer clairement** : `TicketClassifier` plutôt que `Classifier1`
3. **Un module = une tâche** : Gardez les modules focalisés
4. **Composer progressivement** : Testez chaque module individuellement
5. **Documenter** : Ajoutez des docstrings à vos modules personnalisés

### ❌ À éviter

1. **Utiliser ChainOfThought partout** : Plus lent et plus coûteux
2. **Modules trop complexes** : Difficiles à débugger
3. **Trop de composition** : Peut devenir difficile à maintenir
4. **Ignorer les erreurs** : Toujours valider les sorties

### 🎯 Résumé des modules

| Module | Rapidité | Précision | Complexité | Usage |
|--------|----------|-----------|------------|-------|
| **Predict** | ⚡⚡⚡ | 🎯 | 🟢 Simple | Tâches simples, prototypage |
| **ChainOfThought** | ⚡⚡ | 🎯🎯🎯 | 🟢 Simple | Tâches complexes, besoin de raisonnement |
| **ReAct** | ⚡ | 🎯🎯🎯 | 🟡 Moyen | Interactions avec outils |
| **ProgramOfThought** | ⚡⚡ | 🎯🎯🎯🎯 | 🟡 Moyen | Calculs, manipulation de données |
| **Modules composés** | ⚡ | 🎯🎯🎯🎯 | 🔴 Avancé | Pipelines complexes |

---
# Partie 3 : L'évaluation

## Pourquoi évaluer ?

Jusqu'à présent, nous avons créé des modules et observé leurs sorties qualitativement. Mais pour :
- **Comparer** différents modules
- **Mesurer** les améliorations
- **Optimiser** automatiquement (avec GEPA)

...nous avons besoin de **mesures quantitatives** : les **métriques**.

## Qu'est-ce qu'une métrique ?

Une **métrique** est une fonction qui prend :
- Un **exemple** avec la vraie réponse (ground truth)
- Une **prédiction** du modèle
- Et retourne un **score** (généralement entre 0 et 1)

### Format d'une métrique

```python
def ma_metrique(example, prediction, trace=None, pred_name=None, pred_trace=None):
    # Comparer example et prediction
    # Retourner un score entre 0 et 1
    return score
```

**Note** : Les paramètres `trace`, `pred_name` et `pred_trace` sont optionnels et utilisés par certains optimiseurs.

## Métrique 1 : exact match (correspondance exacte)

La métrique la plus stricte : tout doit être parfait.

In [55]:
def exact_match_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    """
    Métrique stricte : 1 si catégorie ET priorité correctes, 0 sinon
    """
    # Normaliser les chaînes (minuscules, sans espaces)
    pred_category = prediction.category.strip().lower()
    true_category = example['category'].strip().lower()
    
    pred_priority = prediction.priority.strip().lower()
    true_priority = example['priority'].strip().lower()
    
    # Les deux doivent être corrects
    if pred_category == true_category and pred_priority == true_priority:
        return 1.0
    else:
        return 0.0

# Test de la métrique
print("🎯 Métrique : exact match")
print("\nTest 1 : Prédiction correcte")
example1 = {"ticket": "Test", "category": "Hardware", "priority": "High"}
pred1 = dspy.Prediction(category="Hardware", priority="High")
score1 = exact_match_metric(example1, pred1)
print(f"  Attendu: Hardware | High")
print(f"  Prédit:  Hardware | High")
print(f"  Score: {score1}")

print("\nTest 2 : Catégorie correcte, priorité incorrecte")
pred2 = dspy.Prediction(category="Hardware", priority="Low")
score2 = exact_match_metric(example1, pred2)
print(f"  Attendu: Hardware | High")
print(f"  Prédit:  Hardware | Low")
print(f"  Score: {score2}")

🎯 Métrique : exact match

Test 1 : Prédiction correcte
  Attendu: Hardware | High
  Prédit:  Hardware | High
  Score: 1.0

Test 2 : Catégorie correcte, priorité incorrecte
  Attendu: Hardware | High
  Prédit:  Hardware | Low
  Score: 0.0


## Métrique 2 : partial match (correspondance partielle)

Plus nuancée : donne des points partiels si au moins un champ est correct.

In [56]:
def partial_match_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    """
    Métrique nuancée avec points partiels :
    - 1.0 : Les deux corrects
    - 0.7 : Catégorie correcte uniquement
    - 0.5 : Priorité correcte uniquement
    - 0.0 : Aucun correct
    """
    pred_category = prediction.category.strip().lower()
    true_category = example['category'].strip().lower()
    
    pred_priority = prediction.priority.strip().lower()
    true_priority = example['priority'].strip().lower()
    
    category_match = (pred_category == true_category)
    priority_match = (pred_priority == true_priority)
    
    if category_match and priority_match:
        return 1.0
    elif category_match:
        return 0.7  # La catégorie est plus importante
    elif priority_match:
        return 0.5
    else:
        return 0.0

# Test de la métrique
print("🎯 Métrique : partial match")
print("\nTest 1 : Les deux corrects")
score1 = partial_match_metric(example1, pred1)
print(f"  Score: {score1}")

print("\nTest 2 : Catégorie correcte uniquement")
score2 = partial_match_metric(example1, pred2)
print(f"  Score: {score2}")

print("\nTest 3 : Priorité correcte uniquement")
pred3 = dspy.Prediction(category="Software", priority="High")
score3 = partial_match_metric(example1, pred3)
print(f"  Score: {score3}")

print("\nTest 4 : Aucun correct")
pred4 = dspy.Prediction(category="Software", priority="Low")
score4 = partial_match_metric(example1, pred4)
print(f"  Score: {score4}")

🎯 Métrique : partial match

Test 1 : Les deux corrects
  Score: 1.0

Test 2 : Catégorie correcte uniquement
  Score: 0.7

Test 3 : Priorité correcte uniquement
  Score: 0.5

Test 4 : Aucun correct
  Score: 0.0


## Fonction d'évaluation réutilisable

Créons une fonction pour évaluer n'importe quel module sur un dataset complet.

In [57]:
def evaluate_module(module, dataset, metric, verbose=False):
    """
    Évalue un module sur un dataset complet
    
    Args:
        module: Le module DSPy à évaluer
        dataset: Liste de dictionnaires avec 'ticket', 'category', 'priority'
        metric: Fonction de métrique
        verbose: Si True, affiche les détails
    
    Returns:
        float: Score moyen (entre 0 et 1)
    """
    total_score = 0
    n_examples = len(dataset)
    
    for i, example in enumerate(dataset):
        # Prédiction
        prediction = module(ticket=example['ticket'])
        
        # Calcul du score
        score = metric(example, prediction)
        total_score += score
        
        # Affichage optionnel
        if verbose:
            print(f"Exemple {i+1}/{n_examples}")
            print(f"  Ticket: {example['ticket'][:50]}...")
            print(f"  Attendu: {example['category']} | {example['priority']}")
            print(f"  Prédit:  {prediction.category} | {prediction.priority}")
            print(f"  Score: {score}\\n")
    
    # Score moyen
    avg_score = total_score / n_examples
    return avg_score

# Test de la fonction d'évaluation
print("📊 Évaluation de ChainOfThought sur le dataset de validation")
print("="*70 + "\\n")

score = evaluate_module(cot_classifier, valset, exact_match_metric, verbose=False)
print(f"Score moyen (exact match): {score:.2%}")

score_partial = evaluate_module(cot_classifier, valset, partial_match_metric, verbose=False)
print(f"Score moyen (partial match): {score_partial:.2%}")

📊 Évaluation de ChainOfThought sur le dataset de validation
======================================================================\n
Score moyen (exact match): 57.14%
Score moyen (partial match): 67.14%


## Comparaison de modules

Maintenant que nous avons des métriques, comparons nos différents modules !

In [58]:
# Comparer tous nos modules
modules_to_compare = [
    ("Predict", predict_classifier),
    ("ChainOfThought", cot_classifier),
    ("Sequential", sequential),
    ("Validated", validated),
    ("Ensemble", ensemble)
]

print("="*70)
print("Comparaison des modules")
print("="*70 + "\\n")

results = []

for name, module in modules_to_compare:
    print(f"Évaluation de {name}...")
    score_exact = evaluate_module(module, valset, exact_match_metric)
    score_partial = evaluate_module(module, valset, partial_match_metric)
    
    results.append({
        'module': name,
        'exact': score_exact,
        'partial': score_partial
    })

print("\\n" + "="*70)
print("Résultats")
print("="*70 + "\\n")

print(f"{'Module':<20} {'Exact Match':<15} {'Partial Match':<15}")
print("-" * 50)
for r in results:
    print(f"{r['module']:<20} {r['exact']:<14.1%} {r['partial']:<14.1%}")

# Trouver le meilleur
best = max(results, key=lambda x: x['exact'])
print(f"\n🏆 Meilleur module (exact match): {best['module']} avec {best['exact']:.1%}")

Comparaison des modules
======================================================================\n
Évaluation de Predict...
Évaluation de ChainOfThought...
Évaluation de Sequential...
Évaluation de Validated...
Évaluation de Ensemble...
\n======================================================================
Résultats
======================================================================\n
Module               Exact Match     Partial Match  
--------------------------------------------------
Predict              57.1%          87.1%         
ChainOfThought       57.1%          67.1%         
Sequential           42.9%          80.0%         
Validated            57.1%          67.1%         
Ensemble             57.1%          67.1%         

🏆 Meilleur module (exact match): Predict avec 57.1%


## 💡 Bonnes pratiques pour l'évaluation

### ✅ À faire

1. **Toujours avoir un dataset de validation séparé** : Ne jamais évaluer sur les données d'entraînement
2. **Utiliser plusieurs métriques** : Exact match + partial match donnent une vue complète
3. **Tester sur des cas limites** : Tickets ambigus, très courts, très longs
4. **Documenter vos métriques** : Expliquez ce que signifie chaque score
5. **Comparer de manière équitable** : Même dataset, même métrique

### ❌ À éviter

1. **Une seule métrique** : Peut cacher des problèmes
2. **Dataset trop petit** : Minimum 20-30 exemples pour validation
3. **Ignorer les erreurs** : Analyser les échecs est crucial
4. **Sur-optimiser** : Attention au surapprentissage sur le dataset de validation

### 📊 Métriques avancées (optionnel)

Pour aller plus loin, vous pouvez calculer :
- **Précision par catégorie** : Performance sur chaque catégorie séparément
- **Matrice de confusion** : Quelles catégories sont confondues
- **Temps d'exécution** : Trade-off précision/vitesse
- **Coût** : Nombre de tokens utilisés

# Partie 4: Les optimiseurs

## 4.1 Introduction: modules vs optimiseurs

Jusqu'à présent, nous avons vu des **modules** (Predict, ChainOfThought, ReAct, etc.). Ces modules **exécutent** des tâches en interrogeant le LLM.

Les **optimiseurs**, quant à eux, **améliorent** les modules en :
- Ajoutant des exemples de démonstration (few-shot learning)
- Optimisant les instructions (prompts)
- Ajustant les paramètres
- Sélectionnant les meilleures configurations

**Analogie** : Si un module est comme un étudiant qui résout un problème, un optimiseur est comme un professeur qui améliore la méthode de l'étudiant en lui montrant des exemples et en affinant ses instructions.

### Principaux optimiseurs DSPy

1. **BootstrapFewShot** - Le plus simple : génère des exemples de démonstration
2. **BootstrapFewShotWithRandomSearch** - Teste plusieurs combinaisons d'exemples
3. **MIPRO** - Optimise à la fois les instructions et les exemples
4. **SignatureOptimizer** - Se concentre uniquement sur l'optimisation des instructions
5. **GEPA** - Le plus sophistiqué : utilise des algorithmes génétiques et la réflexion LLM

Dans cette section, nous allons explorer les optimiseurs 1-4. GEPA sera couvert en détail dans la Partie 7.

## 4.2 BootstrapFewShot: générer des exemples de démonstration

**BootstrapFewShot** est l'optimiseur le plus simple de DSPy. Il fonctionne en :

1. Exécutant votre module sur les données d'entraînement
2. Gardant les prédictions correctes (validées par votre métrique)
3. Utilisant ces prédictions comme exemples de démonstration (few-shot)
4. Injectant ces exemples dans le prompt du module optimisé

**Avantages** :
- Simple à comprendre et à utiliser
- Rapide à exécuter
- Amélioration typique de 5-15%

**Inconvénients** :
- Ne modifie pas les instructions
- Qualité dépend de la qualité des données d'entraînement

### Exemple pratique

In [59]:
from dspy.teleprompt import BootstrapFewShot

print("🔧 Optimisation avec BootstrapFewShot...")

# 1. Préparer les données au format DSPy
train_examples = [
    dspy.Example(
        ticket=ex['ticket'],
        category=ex['category'],
        priority=ex['priority']
    ).with_inputs('ticket')
    for ex in trainset
]

# 2. Créer le module de base
basic_classifier = SimpleTicketClassifier()

# 3. Évaluer AVANT optimisation
print("\n📊 Performance AVANT optimisation:")
score_before = evaluate_module(basic_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_before:.2%}")

# 4. Configurer l'optimiseur BootstrapFewShot
optimizer = BootstrapFewShot(
    metric=exact_match_metric,
    max_bootstrapped_demos=4,  # Nombre d'exemples à générer
    max_labeled_demos=4         # Nombre max d'exemples à utiliser
)

# 5. Compiler (optimiser) le module
print("\n🔄 Compilation avec BootstrapFewShot...")
optimized_classifier = optimizer.compile(
    student=basic_classifier,
    trainset=train_examples
)

# 6. Évaluer APRÈS optimisation
print("\n📊 Performance APRÈS optimisation:")
score_after = evaluate_module(optimized_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_after:.2%}")

# 7. Calculer l'amélioration
improvement = ((score_after - score_before) / score_before) * 100 if score_before > 0 else 0
print(f"\n📈 Amélioration: {improvement:+.1f}%")

🔧 Optimisation avec BootstrapFewShot...


NameError: name 'SimpleTicketClassifier' is not defined

### Inspecter les exemples générés

BootstrapFewShot a ajouté des exemples de démonstration au module. Nous pouvons les visualiser :

In [ ]:
# Inspecter les exemples de démonstration ajoutés par BootstrapFewShot
if hasattr(optimized_classifier, 'classifier'):
    predictor = optimized_classifier.classifier
    
    if hasattr(predictor, 'demos') and predictor.demos:
        print(f"📚 BootstrapFewShot a ajouté {len(predictor.demos)} exemples de démonstration:\n")
        
        for i, demo in enumerate(predictor.demos[:3], 1):  # Afficher les 3 premiers
            print(f"Exemple {i}:")
            print(f"  Ticket: {demo.ticket[:80]}...")
            print(f"  Catégorie: {demo.category}")
            print(f"  Priorité: {demo.priority}")
            print()
    else:
        print("ℹ️ Aucun exemple de démonstration trouvé (ou version DSPy différente)")
else:
    print("ℹ️ Structure du module différente de celle attendue")

## 4.3 MIPRO: optimisation des instructions et exemples

**MIPRO** (Multi-prompt Instruction Proposal Optimizer) est un optimiseur plus avancé qui :

1. **Génère plusieurs variantes d'instructions** pour votre signature
2. **Sélectionne les meilleurs exemples** de démonstration
3. **Teste différentes combinaisons** (instructions × exemples)
4. **Garde la meilleure configuration** selon votre métrique

**Avantages** :
- Optimise à la fois les instructions et les exemples
- Amélioration typique de 10-25%
- Explore l'espace des possibilités de manière systématique

**Inconvénients** :
- Plus lent que BootstrapFewShot (nécessite plus d'appels LLM)
- Peut prendre 10-20 minutes avec Ollama local

### Exemple pratique

In [ ]:
from dspy.teleprompt import MIPRO

print("🚀 Optimisation avec MIPRO...")
print("⏰ Attention : Cela peut prendre 10-20 minutes avec Ollama\n")

# 1. Créer un module de base
mipro_classifier = SimpleTicketClassifier()

# 2. Configurer MIPRO
optimizer = MIPRO(
    metric=exact_match_metric,
    num_candidates=5,           # Nombre de variantes d'instructions à générer
    init_temperature=1.0        # Température pour la génération d'instructions
)

# 3. Compiler avec MIPRO
# Note: MIPRO nécessite à la fois trainset et valset (optionnel)
print("🔄 Compilation avec MIPRO (cela va prendre du temps)...")

try:
    mipro_optimized = optimizer.compile(
        student=mipro_classifier,
        trainset=train_examples,
        max_bootstrapped_demos=3,  # Nombre d'exemples à générer
        max_labeled_demos=3,        # Nombre max d'exemples à utiliser
        requires_permission_to_run=False  # Pas de confirmation interactive
    )
    
    # 4. Évaluer le résultat
    print("\n📊 Performance après MIPRO:")
    score_mipro = evaluate_module(mipro_optimized, val_examples, exact_match_metric)
    print(f"   Score: {score_mipro:.2%}")
    
    improvement_mipro = ((score_mipro - score_before) / score_before) * 100 if score_before > 0 else 0
    print(f"   Amélioration vs baseline: {improvement_mipro:+.1f}%")
    
except Exception as e:
    print(f"⚠️ Erreur lors de l'optimisation MIPRO: {e}")
    print("   MIPRO nécessite une configuration spécifique selon la version de DSPy")
    print("   Consultez la documentation DSPy pour plus de détails.")

## 4.4 Autres optimiseurs importants

### SignatureOptimizer

**SignatureOptimizer** se concentre uniquement sur l'optimisation des instructions de votre signature, sans ajouter d'exemples de démonstration.

**Utilisation** :
```python
from dspy.teleprompt import SignatureOptimizer

optimizer = SignatureOptimizer(
    metric=exact_match_metric,
    breadth=10,  # Nombre de variantes à générer
    depth=3      # Nombre d'itérations de raffinement
)

optimized = optimizer.compile(student=classifier, trainset=train_examples)
```

**Avantages** :
- Rapide (pas de génération d'exemples)
- Améliore la clarté des instructions
- Bon pour les tâches où les exemples ne sont pas critiques

**Inconvénients** :
- Amélioration modeste (5-10%)
- Ne tire pas parti du few-shot learning

### BootstrapFewShotWithRandomSearch

Extension de BootstrapFewShot qui teste plusieurs combinaisons aléatoires d'exemples.

**Utilisation** :
```python
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(
    metric=exact_match_metric,
    max_bootstrapped_demos=4,
    num_candidate_programs=8  # Nombre de combinaisons à tester
)

optimized = optimizer.compile(student=classifier, trainset=train_examples)
```

**Avantages** :
- Trouve de meilleures combinaisons d'exemples
- Plus robuste que BootstrapFewShot simple
- Amélioration typique de 8-18%

**Inconvénients** :
- Plus lent que BootstrapFewShot (teste plusieurs combinaisons)

## 4.5 Comparaison des optimiseurs

| Optimiseur | Ce qu'il optimise | Vitesse | Amélioration typique | Complexité | Quand l'utiliser |
|------------|------------------|---------|---------------------|------------|------------------|
| **BootstrapFewShot** | Exemples uniquement | ⚡⚡⚡ Rapide | 5-15% | Simple | Première optimisation, tests rapides |
| **BootstrapFewShotWithRandomSearch** | Exemples (avec recherche) | ⚡⚡ Moyen | 8-18% | Simple | Quand vous avez un peu plus de temps |
| **SignatureOptimizer** | Instructions uniquement | ⚡⚡⚡ Rapide | 5-10% | Moyen | Tâches où les instructions sont critiques |
| **MIPRO** | Instructions + exemples | ⚡ Lent | 10-25% | Avancé | Production, quand vous visez la meilleure performance |
| **GEPA** | Instructions + exemples + réflexion | ⚡ Très lent | 15-30% | Très avancé | Maximum de performance, recherche |

### Guide de sélection

**Pour débuter** :
1. Commencez avec **BootstrapFewShot** pour comprendre le concept
2. Si les résultats sont bons, passez à **BootstrapFewShotWithRandomSearch**
3. Si vous avez du temps, essayez **MIPRO**

**Pour la production** :
- Si vous avez des contraintes de temps : **BootstrapFewShotWithRandomSearch**
- Si vous visez la meilleure performance : **MIPRO** ou **GEPA**
- Si vos instructions sont mal formulées : **SignatureOptimizer** d'abord, puis un autre

**Pour la recherche** :
- **GEPA** pour explorer les limites de performance
- **MIPRO** pour une approche plus systématique

### Stratégie recommandée

1. **Phase 1 - Exploration** : Utilisez BootstrapFewShot pour tester rapidement
2. **Phase 2 - Amélioration** : Passez à MIPRO si les résultats sont prometteurs
3. **Phase 3 - Optimisation finale** : Utilisez GEPA pour maximiser la performance (voir Partie 7)

## 4.6 Introduction à GEPA

**GEPA** (Genetic-Pareto Algorithm) est l'optimiseur le plus sophistiqué de DSPy. Il mérite une section complète (Partie 7) car il combine plusieurs techniques avancées :

### Comment GEPA fonctionne

1. **Algorithmes génétiques** : Évolution de populations de prompts
2. **Réflexion LLM** : Utilise un LLM pour analyser les erreurs et proposer des améliorations
3. **Optimisation Pareto** : Équilibre plusieurs objectifs (précision, concision, etc.)
4. **Itérations adaptatives** : Apprend de ses erreurs pour s'améliorer

### Nouveautés dans DSPy 3.0+

GEPA a été intégré directement dans DSPy et nécessite maintenant :
- Un **reflection_lm** : modèle dédié à l'analyse des erreurs
- Configuration via **auto** : 'light', 'medium', ou 'heavy'
- Métrique compatible avec les nouveaux paramètres

### Aperçu rapide

```python
from dspy.teleprompt import GEPA

# Configuration du modèle de réflexion
reflection_lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=1.0,
    max_tokens=8000
)

# Optimiseur GEPA
optimizer = GEPA(
    metric=exact_match_metric,
    auto='light',  # 'light', 'medium', ou 'heavy'
    reflection_lm=reflection_lm
)

# Compilation
optimized = optimizer.compile(
    student=classifier,
    trainset=train_examples,
    valset=val_examples
)
```

### Quand utiliser GEPA

- ✅ Vous avez du temps (15-30 minutes minimum)
- ✅ Vous visez la meilleure performance possible
- ✅ Vous avez suffisamment de données d'entraînement (20+ exemples)
- ✅ Votre métrique est bien définie
- ✅ Vous êtes prêt à expérimenter avec les paramètres

**La Partie 7 de ce tutoriel couvre GEPA en profondeur avec des exemples pratiques, des comparaisons de paramètres, et des conseils d'optimisation.**

## 4.7 Résumé de la Partie 4

### Ce que nous avons appris

1. **Différence modules vs optimiseurs** :
   - Les modules **exécutent** des tâches
   - Les optimiseurs **améliorent** les modules

2. **Les optimiseurs principaux** :
   - **BootstrapFewShot** : Simple, rapide, génère des exemples
   - **BootstrapFewShotWithRandomSearch** : Teste plusieurs combinaisons
   - **SignatureOptimizer** : Optimise uniquement les instructions
   - **MIPRO** : Optimise instructions + exemples
   - **GEPA** : Le plus sophistiqué (détails en Partie 7)

3. **Comment choisir** :
   - Débuter : BootstrapFewShot
   - Production rapide : BootstrapFewShotWithRandomSearch
   - Meilleure performance : MIPRO ou GEPA
   - Instructions mal formulées : SignatureOptimizer

### Points clés à retenir

- ✅ Toujours évaluer **avant** et **après** optimisation
- ✅ Commencer simple, puis complexifier
- ✅ Les optimiseurs nécessitent des données d'entraînement de qualité
- ✅ Plus de temps d'optimisation = généralement meilleure performance
- ✅ Votre métrique détermine ce que l'optimiseur optimise

### Prochaines étapes

- **Partie 5** : Multi-modèles (utiliser différents LLMs)
- **Partie 6** : Patterns avancés (optionnel)
- **Partie 7** : GEPA en pratique (optimisation approfondie)

# Partie 5: Multi-modèles et flexibilité

## 5.1 Introduction: pourquoi utiliser plusieurs modèles?

DSPy offre une **abstraction puissante** : votre code reste le même quel que soit le modèle utilisé. Vous pouvez :

1. **Changer de fournisseur** facilement (Ollama → OpenAI → Anthropic)
2. **Comparer les performances** de différents modèles
3. **Créer des architectures hybrides** (modèle rapide pour catégorie, modèle précis pour priorité)
4. **Optimiser coût vs performance**

### Avantages du multi-modèles

- 🔄 **Flexibilité** : Pas de vendor lock-in
- 💰 **Optimisation des coûts** : Modèle gratuit (Ollama) pour dev, modèle payant pour prod
- 🎯 **Performance** : Choisir le meilleur modèle pour chaque tâche
- 🧪 **Expérimentation** : Tester facilement différents modèles

### Ce que nous allons voir

1. Configurer différents fournisseurs (Ollama, OpenAI, Anthropic)
2. Comparer les performances de différents modèles
3. Créer des architectures hybrides
4. Gérer les clés API de manière sécurisée

## 5.2 Configuration de différents fournisseurs

### 5.2.1 Ollama (local, gratuit)

Ollama permet d'exécuter des modèles **localement** sans API key ni coûts.

**Modèles recommandés** :
- `llama3.1:8b` - Équilibré, bon pour la plupart des tâches (4.7 GB)
- `mistral:7b` - Rapide, bon pour les tâches simples (4.1 GB)
- `qwen2.5:7b` - Haute qualité, excellent pour les tâches complexes (4.7 GB)
- `gemma2:9b` - Alternative de Google, très performant (5.4 GB)

**Configuration** :

In [ ]:
# Configuration Ollama
lm_ollama_llama = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

lm_ollama_mistral = dspy.LM(
    model='ollama_chat/mistral:7b',
    api_base='http://localhost:11434',
    temperature=0.3
)

lm_ollama_qwen = dspy.LM(
    model='ollama_chat/qwen2.5:7b',
    api_base='http://localhost:11434',
    temperature=0.3
)

print("✅ Modèles Ollama configurés:")
print("   - llama3.1:8b")
print("   - mistral:7b")
print("   - qwen2.5:7b")

### 5.2.2 OpenAI (API, payant)

OpenAI propose des modèles très performants via API.

**Modèles recommandés** :
- `gpt-4o-mini` - Rapide et économique, bon rapport qualité/prix
- `gpt-4o` - Haute performance, multimodal
- `gpt-4-turbo` - Équilibré performance/coût

**Configuration** :

In [ ]:
import os

# Configuration OpenAI (nécessite OPENAI_API_KEY dans l'environnement)
if os.getenv('OPENAI_API_KEY'):
    lm_openai_mini = dspy.LM(
        model='openai/gpt-4o-mini',
        temperature=0.3
    )
    
    lm_openai_4o = dspy.LM(
        model='openai/gpt-4o',
        temperature=0.3
    )
    
    print("✅ Modèles OpenAI configurés:")
    print("   - gpt-4o-mini")
    print("   - gpt-4o")
else:
    print("⚠️ OPENAI_API_KEY non définie - modèles OpenAI non disponibles")
    print("   Pour utiliser OpenAI, définissez la variable d'environnement:")
    print("   export OPENAI_API_KEY='votre-clé-api'")
    lm_openai_mini = None
    lm_openai_4o = None

### 5.2.3 Anthropic (API, payant)

Anthropic propose les modèles Claude, connus pour leur qualité et leur sécurité.

**Modèles recommandés** :
- `claude-3-5-haiku-20241022` - Rapide et économique
- `claude-3-5-sonnet-20241022` - Équilibré, excellent pour la plupart des tâches
- `claude-3-opus-20240229` - Maximum de performance

**Configuration** :

In [ ]:
# Configuration Anthropic (nécessite ANTHROPIC_API_KEY dans l'environnement)
if os.getenv('ANTHROPIC_API_KEY'):
    lm_claude_haiku = dspy.LM(
        model='anthropic/claude-3-5-haiku-20241022',
        temperature=0.3
    )
    
    lm_claude_sonnet = dspy.LM(
        model='anthropic/claude-3-5-sonnet-20241022',
        temperature=0.3
    )
    
    print("✅ Modèles Anthropic configurés:")
    print("   - claude-3-5-haiku")
    print("   - claude-3-5-sonnet")
else:
    print("⚠️ ANTHROPIC_API_KEY non définie - modèles Anthropic non disponibles")
    print("   Pour utiliser Anthropic, définissez la variable d'environnement:")
    print("   export ANTHROPIC_API_KEY='votre-clé-api'")
    lm_claude_haiku = None
    lm_claude_sonnet = None

## 5.3 Comparer les performances de différents modèles

Maintenant que nous avons configuré plusieurs modèles, comparons leurs performances sur notre tâche de classification de tickets IT.

### Fonction de benchmarking

In [ ]:
import time

def benchmark_model(lm, model_name, examples, metric):
    """
    Évalue un modèle sur un ensemble d'exemples
    
    Args:
        lm: Le language model DSPy
        model_name: Nom du modèle (pour affichage)
        examples: Liste d'exemples de validation
        metric: Fonction de métrique
    
    Returns:
        dict avec score et temps d'exécution
    """
    # Configurer DSPy avec ce modèle
    dspy.configure(lm=lm)
    
    # Créer un classifier avec ce modèle
    classifier = SimpleTicketClassifier()
    
    # Mesurer le temps
    start_time = time.time()
    
    # Évaluer
    total_score = 0
    for example in examples:
        prediction = classifier(ticket=example['ticket'])
        score = metric(example, prediction)
        total_score += score
    
    end_time = time.time()
    
    # Calculer les résultats
    avg_score = total_score / len(examples)
    elapsed_time = end_time - start_time
    
    return {
        'model': model_name,
        'score': avg_score,
        'time': elapsed_time
    }

print("✅ Fonction de benchmarking définie")

### Comparaison des modèles Ollama

Comparons les 3 modèles Ollama locaux :

In [ ]:
print("🔍 Comparaison des modèles Ollama...")
print("⏰ Cela va prendre quelques minutes\n")

results = []

# Benchmarker llama3.1:8b
print("1/3 Évaluation de llama3.1:8b...")
result_llama = benchmark_model(lm_ollama_llama, 'llama3.1:8b', valset, exact_match_metric)
results.append(result_llama)
print(f"   Score: {result_llama['score']:.2%} | Temps: {result_llama['time']:.1f}s\n")

# Benchmarker mistral:7b
print("2/3 Évaluation de mistral:7b...")
result_mistral = benchmark_model(lm_ollama_mistral, 'mistral:7b', valset, exact_match_metric)
results.append(result_mistral)
print(f"   Score: {result_mistral['score']:.2%} | Temps: {result_mistral['time']:.1f}s\n")

# Benchmarker qwen2.5:7b
print("3/3 Évaluation de qwen2.5:7b...")
result_qwen = benchmark_model(lm_ollama_qwen, 'qwen2.5:7b', valset, exact_match_metric)
results.append(result_qwen)
print(f"   Score: {result_qwen['score']:.2%} | Temps: {result_qwen['time']:.1f}s\n")

# Afficher le résumé
print("=" * 60)
print("📊 RÉSUMÉ DES PERFORMANCES")
print("=" * 60)
for r in sorted(results, key=lambda x: x['score'], reverse=True):
    print(f"{r['model']:20} | Score: {r['score']:6.2%} | Temps: {r['time']:5.1f}s")
print("=" * 60)

## 5.4 Architectures hybrides: utiliser différents modèles pour différentes tâches

Une **architecture hybride** utilise différents modèles pour différentes parties de votre pipeline. Par exemple :

- Modèle **rapide et économique** pour la catégorisation
- Modèle **précis mais coûteux** pour la priorisation

### Avantages

- 💰 **Optimisation des coûts** : Utiliser des modèles coûteux uniquement quand nécessaire
- ⚡ **Optimisation de la vitesse** : Modèles rapides pour les tâches simples
- 🎯 **Optimisation de la qualité** : Meilleurs modèles pour les tâches critiques

### Exemple: pipeline hybride

Créons un classifier qui utilise deux modèles différents :

In [ ]:
class HybridTicketClassifier(dspy.Module):
    """
    Classifier hybride utilisant 2 modèles différents:
    - Modèle rapide pour la catégorie
    - Modèle précis pour la priorité
    """
    
    def __init__(self, fast_lm, accurate_lm):
        super().__init__()
        self.fast_lm = fast_lm
        self.accurate_lm = accurate_lm
        
        # Signature pour la catégorie
        self.category_signature = CategoryClassifier
        
        # Signature pour la priorité
        self.priority_signature = PriorityClassifier
    
    def forward(self, ticket):
        # Étape 1: Catégorisation avec le modèle rapide
        with dspy.settings.context(lm=self.fast_lm):
            category_predictor = dspy.ChainOfThought(self.category_signature)
            category_result = category_predictor(ticket=ticket)
        
        # Étape 2: Priorisation avec le modèle précis
        with dspy.settings.context(lm=self.accurate_lm):
            priority_predictor = dspy.ChainOfThought(self.priority_signature)
            priority_result = priority_predictor(
                ticket=ticket,
                category=category_result.category
            )
        
        return dspy.Prediction(
            category=category_result.category,
            priority=priority_result.priority
        )

print("✅ Classe HybridTicketClassifier définie")

### Tester le classifier hybride

In [ ]:
print("🔀 Test du classifier hybride")
print("   Modèle rapide (catégorie): mistral:7b")
print("   Modèle précis (priorité): llama3.1:8b\n")

# Créer le classifier hybride
hybrid_classifier = HybridTicketClassifier(
    fast_lm=lm_ollama_mistral,      # Rapide pour la catégorie
    accurate_lm=lm_ollama_llama     # Précis pour la priorité
)

# Configurer DSPy (requis pour l'exécution)
dspy.configure(lm=lm_ollama_llama)

# Tester sur quelques exemples
print("📝 Exemples de prédictions:\n")

test_tickets = [
    "Mon ordinateur portable ne démarre plus, j'ai une présentation importante dans 2 heures",
    "Je voudrais accès au VPN pour le télétravail quand c'est possible",
    "Toutes les imprimantes de l'étage sont hors ligne"
]

for i, ticket in enumerate(test_tickets, 1):
    result = hybrid_classifier(ticket=ticket)
    print(f"{i}. Ticket: {ticket[:60]}...")
    print(f"   → Catégorie: {result.category} | Priorité: {result.priority}\n")

# Évaluer sur l'ensemble de validation
print("📊 Évaluation sur l'ensemble de validation:")
score_hybrid = evaluate_module(hybrid_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_hybrid:.2%}")

## 5.5 Guide de sélection de modèles

### Critères de sélection

| Critère | Ollama (local) | OpenAI | Anthropic |
|---------|---------------|--------|-----------|
| **Coût** | Gratuit (matériel local) | Payant à l'usage | Payant à l'usage |
| **Vitesse** | Dépend du matériel | Rapide (API cloud) | Rapide (API cloud) |
| **Confidentialité** | ✅ 100% local | ⚠️ Données envoyées à OpenAI | ⚠️ Données envoyées à Anthropic |
| **Qualité** | Bonne (7-8B params) | Excellente | Excellente |
| **Disponibilité** | Nécessite installation | API toujours disponible | API toujours disponible |
| **Latence** | Faible (local) | Moyenne (réseau) | Moyenne (réseau) |

### Recommandations par cas d'usage

#### 1. Développement et prototypage
**Choix recommandé** : Ollama (llama3.1:8b ou qwen2.5:7b)
- ✅ Gratuit, itérations rapides
- ✅ Pas de limite de requêtes
- ✅ Confidentialité des données

#### 2. Production à faible volume (<1000 requêtes/jour)
**Choix recommandé** : OpenAI (gpt-4o-mini) ou Anthropic (claude-3-5-haiku)
- ✅ Coûts acceptables
- ✅ Haute disponibilité
- ✅ Excellente qualité

#### 3. Production à haut volume (>10000 requêtes/jour)
**Choix recommandé** : Architecture hybride
- Ollama pour les tâches simples (catégorisation)
- API payante pour les tâches critiques (priorisation)
- ✅ Optimisation du rapport coût/performance

#### 4. Données sensibles (santé, finance, etc.)
**Choix recommandé** : Ollama uniquement
- ✅ Aucune donnée ne quitte votre infrastructure
- ✅ Conformité RGPD/HIPAA facilitée

#### 5. Recherche et expérimentation
**Choix recommandé** : Tous les modèles
- Tester plusieurs modèles pour trouver le meilleur
- Utiliser Ollama pour les itérations rapides
- Valider avec des modèles API avant production

## 5.6 Résumé de la Partie 5

### Ce que nous avons appris

1. **Abstraction DSPy** : Un seul code fonctionne avec tous les fournisseurs LLM
   
2. **Configuration de fournisseurs** :
   - **Ollama** : Local, gratuit, confidentialité maximale
   - **OpenAI** : API cloud, haute qualité, payant
   - **Anthropic** : API cloud, excellente qualité, payant

3. **Comparaison de modèles** :
   - Fonction de benchmarking pour comparer performances
   - Mesure du score ET du temps d'exécution
   - Aide à la décision data-driven

4. **Architectures hybrides** :
   - Différents modèles pour différentes tâches
   - Optimisation coût/performance/vitesse
   - Utilisation de `dspy.settings.context(lm=...)`

5. **Guide de sélection** :
   - Critères : coût, vitesse, confidentialité, qualité
   - Recommandations par cas d'usage
   - Stratégies adaptées au contexte

### Points clés à retenir

- ✅ **Flexibilité** : Changer de modèle ne nécessite que quelques lignes de code
- ✅ **Expérimentation** : Tester plusieurs modèles est facile et recommandé
- ✅ **Optimisation** : Architectures hybrides pour le meilleur rapport coût/performance
- ✅ **Confidentialité** : Ollama pour les données sensibles
- ✅ **Évolutivité** : Commencer avec Ollama, migrer vers API si nécessaire

### Exemple de workflow recommandé

```python
# 1. Développement avec Ollama (gratuit, rapide)
lm_dev = dspy.LM('ollama_chat/llama3.1:8b', api_base='http://localhost:11434')
dspy.configure(lm=lm_dev)

# 2. Test avec plusieurs modèles
models = [lm_ollama_llama, lm_ollama_qwen, lm_openai_mini]
results = [benchmark_model(lm, name, valset, metric) for lm, name in models]

# 3. Production avec le meilleur modèle ou architecture hybride
lm_prod = best_model  # Ou HybridTicketClassifier(fast_lm, accurate_lm)
```

### Prochaines étapes

- **Partie 6** : Patterns avancés (optionnel - validation, retry, fallback)
- **Partie 7** : GEPA en pratique (optimisation sophistiquée)

# Partie 6: Patterns avancés (optionnel)

## 6.1 Introduction aux patterns avancés

Cette section couvre des **patterns de production** pour rendre vos modules DSPy plus robustes, fiables et performants.

### Pourquoi utiliser ces patterns?

En production, les LLMs peuvent :
- ❌ Générer des sorties invalides (mauvais format, valeurs hors limites)
- ❌ Échouer temporairement (timeout, rate limiting)
- ❌ Produire des résultats incohérents
- ❌ Être indisponibles (downtime API)

Les **patterns avancés** permettent de gérer ces situations.

### Patterns couverts

1. **Validation** : Vérifier et corriger les sorties invalides
2. **Retry** : Réessayer automatiquement en cas d'erreur
3. **Fallback** : Utiliser un modèle de secours si le principal échoue
4. **Ensemble** : Combiner plusieurs prédictions pour plus de robustesse
5. **Chaînage avec contrôle** : Pipelines complexes avec branchement conditionnel

Ces patterns sont **optionnels** mais fortement recommandés pour les applications de production.

## 6.2 Pattern de validation

Le **pattern de validation** vérifie que les sorties du LLM respectent les contraintes de votre application.

### Problème

Les LLMs peuvent générer :
- Des catégories qui n'existent pas ("Matériel" au lieu de "Hardware")
- Des priorités invalides ("Très urgent" au lieu de "Urgent")
- Des formats incorrects (minuscules au lieu de majuscules)

### Solution

Créer un module qui :
1. Exécute la prédiction
2. **Valide** la sortie
3. **Corrige** ou **rejette** si invalide
4. Optionnellement, **réessaye** avec des instructions clarifiées

### Exemple: classifier avec validation

In [ ]:
class ValidatedTicketClassifier(dspy.Module):
    """
    Classifier avec validation des sorties
    """
    
    def __init__(self):
        super().__init__()
        self.classifier = dspy.ChainOfThought(TicketClassifier)
        
        # Valeurs valides
        self.valid_categories = set(cat.lower() for cat in CATEGORIES)
        self.valid_priorities = set(pri.lower() for pri in PRIORITIES)
    
    def validate_and_correct(self, category, priority):
        """
        Valide et corrige les sorties si nécessaire
        
        Returns:
            (category, priority, is_valid)
        """
        category_lower = category.strip().lower()
        priority_lower = priority.strip().lower()
        
        is_valid = True
        
        # Validation de la catégorie
        if category_lower not in self.valid_categories:
            # Essayer de trouver une correspondance approximative
            if 'hard' in category_lower or 'matér' in category_lower:
                category = 'Hardware'
            elif 'soft' in category_lower or 'logic' in category_lower:
                category = 'Software'
            elif 'réseau' in category_lower or 'network' in category_lower:
                category = 'Network'
            elif 'compte' in category_lower or 'account' in category_lower:
                category = 'Account'
            else:
                # Par défaut, mettre "Other"
                category = 'Other'
                is_valid = False
        else:
            # Normaliser la casse
            category = next(c for c in CATEGORIES if c.lower() == category_lower)
        
        # Validation de la priorité
        if priority_lower not in self.valid_priorities:
            # Essayer de mapper
            if 'critic' in priority_lower or 'critique' in priority_lower:
                priority = 'Critical'
            elif 'urgent' in priority_lower:
                priority = 'Urgent'
            elif 'high' in priority_lower or 'haut' in priority_lower:
                priority = 'High'
            elif 'medium' in priority_lower or 'moyen' in priority_lower:
                priority = 'Medium'
            else:
                priority = 'Low'
                is_valid = False
        else:
            # Normaliser la casse
            priority = next(p for p in PRIORITIES if p.lower() == priority_lower)
        
        return category, priority, is_valid
    
    def forward(self, ticket):
        # Prédiction initiale
        result = self.classifier(ticket=ticket)
        
        # Validation et correction
        category, priority, is_valid = self.validate_and_correct(
            result.category,
            result.priority
        )
        
        return dspy.Prediction(
            category=category,
            priority=priority,
            is_valid=is_valid
        )

print("✅ Classe ValidatedTicketClassifier définie")

### Test du classifier avec validation

In [ ]:
print("✅ Test du classifier avec validation\n")

# Configurer le modèle
dspy.configure(lm=lm_ollama_llama)

# Créer le classifier validé
validated_classifier = ValidatedTicketClassifier()

# Tester sur quelques exemples
test_examples = [
    "Mon imprimante ne fonctionne pas",
    "Besoin d'accès au serveur de fichiers",
    "Le réseau est très lent, urgent!"
]

for i, ticket in enumerate(test_examples, 1):
    result = validated_classifier(ticket=ticket)
    status = "✅ Valide" if result.is_valid else "⚠️ Corrigé"
    print(f"{i}. {ticket}")
    print(f"   → {result.category} | {result.priority} | {status}\n")

# Évaluer sur l'ensemble de validation
print("📊 Évaluation sur l'ensemble de validation:")
score = evaluate_module(validated_classifier, val_examples, exact_match_metric)
print(f"   Score: {score:.2%}")

# Compter les corrections effectuées
corrections = 0
for example in val_examples[:10]:
    result = validated_classifier(ticket=example['ticket'])
    if not result.is_valid:
        corrections += 1

print(f"   Corrections appliquées: {corrections}/10 sur les premiers exemples")

## 6.3 Pattern de retry (réessayer en cas d'erreur)

Le **pattern de retry** réessaye automatiquement une opération en cas d'échec temporaire.

### Problème

Les APIs LLM peuvent échouer pour diverses raisons :
- Timeout réseau
- Rate limiting (trop de requêtes)
- Surcharge temporaire du service
- Erreurs intermittentes

### Solution

Implémenter une logique de retry avec :
1. **Nombre maximum de tentatives** (ex: 3)
2. **Délai exponentiel** entre les tentatives (1s, 2s, 4s)
3. **Gestion des erreurs** spécifiques

### Exemple: classifier avec retry

In [ ]:
import time

class RetryTicketClassifier(dspy.Module):
    """
    Classifier avec logique de retry en cas d'erreur
    """
    
    def __init__(self, max_retries=3, initial_delay=1.0):
        super().__init__()
        self.classifier = dspy.ChainOfThought(TicketClassifier)
        self.max_retries = max_retries
        self.initial_delay = initial_delay
    
    def forward(self, ticket):
        last_error = None
        
        for attempt in range(self.max_retries):
            try:
                # Tentative de prédiction
                result = self.classifier(ticket=ticket)
                
                # Succès - retourner le résultat
                return dspy.Prediction(
                    category=result.category,
                    priority=result.priority,
                    attempts=attempt + 1
                )
                
            except Exception as e:
                last_error = e
                
                # Si c'est la dernière tentative, on va lever l'erreur
                if attempt == self.max_retries - 1:
                    break
                
                # Calculer le délai avec backoff exponentiel
                delay = self.initial_delay * (2 ** attempt)
                
                print(f"⚠️ Tentative {attempt + 1} échouée: {e}")
                print(f"   Nouvelle tentative dans {delay:.1f}s...")
                
                time.sleep(delay)
        
        # Toutes les tentatives ont échoué
        raise Exception(f"Échec après {self.max_retries} tentatives: {last_error}")

print("✅ Classe RetryTicketClassifier définie")

## 6.4 Pattern de fallback (modèle de secours)

Le **pattern de fallback** utilise un modèle de secours si le modèle principal échoue.

### Problème

- Le modèle principal (API payante) peut être indisponible
- Vous voulez garantir la disponibilité du service
- Budget limité : utiliser un modèle local en secours

### Solution

Créer un module qui :
1. Essaye avec le **modèle principal** (haute qualité)
2. Si échec, bascule vers le **modèle de secours** (local/gratuit)
3. Retourne quelle stratégie a été utilisée

### Exemple: classifier avec fallback

In [ ]:
class FallbackTicketClassifier(dspy.Module):
    """
    Classifier avec fallback vers un modèle de secours
    """
    
    def __init__(self, primary_lm, fallback_lm):
        super().__init__()
        self.primary_lm = primary_lm
        self.fallback_lm = fallback_lm
        self.signature = TicketClassifier
    
    def forward(self, ticket):
        # Tentative avec le modèle principal
        try:
            with dspy.settings.context(lm=self.primary_lm):
                predictor = dspy.ChainOfThought(self.signature)
                result = predictor(ticket=ticket)
                
                return dspy.Prediction(
                    category=result.category,
                    priority=result.priority,
                    model_used='primary'
                )
        
        except Exception as e:
            print(f"⚠️ Modèle principal échoué: {e}")
            print(f"   Basculement vers le modèle de secours...")
            
            # Fallback vers le modèle de secours
            try:
                with dspy.settings.context(lm=self.fallback_lm):
                    predictor = dspy.ChainOfThought(self.signature)
                    result = predictor(ticket=ticket)
                    
                    return dspy.Prediction(
                        category=result.category,
                        priority=result.priority,
                        model_used='fallback'
                    )
            
            except Exception as fallback_error:
                # Les deux modèles ont échoué
                raise Exception(f"Échec des deux modèles. Fallback error: {fallback_error}")

print("✅ Classe FallbackTicketClassifier définie")

### Test du classifier avec fallback

In [ ]:
print("🔀 Test du classifier avec fallback")
print("   Modèle principal: llama3.1:8b")
print("   Modèle de secours: mistral:7b\n")

# Créer le classifier avec fallback
fallback_classifier = FallbackTicketClassifier(
    primary_lm=lm_ollama_llama,
    fallback_lm=lm_ollama_mistral
)

# Tester (normalement, le modèle principal devrait fonctionner)
test_ticket = "Mon imprimante ne fonctionne plus"
result = fallback_classifier(ticket=test_ticket)

print(f"Ticket: {test_ticket}")
print(f"Résultat: {result.category} | {result.priority}")
print(f"Modèle utilisé: {result.model_used}")
print()

# Note: Dans un cas réel, le fallback se déclencherait si:
# - L'API est indisponible
# - Le rate limit est atteint
# - Le timeout est dépassé
# - Une erreur réseau se produit

print("💡 Le pattern de fallback garantit la disponibilité du service")
print("   même en cas de problème avec le modèle principal.")

## 6.5 Pattern de raffinement itératif (Refine)

`dspy.Refine` est un module qui améliore itérativement une prédiction en :
1. **Exécutant le module N fois** avec différents paramètres
2. **Évaluant chaque prédiction** avec une fonction de récompense
3. **Retournant la meilleure prédiction** ou s'arrêtant si le seuil est atteint

### Différence avec les autres modules :

| Module | Approche |
|--------|----------|
| `ChainOfThought` | Une seule tentative avec raisonnement |
| `ReAct` | Raisonnement + actions séquentielles |
| `Refine` | **Plusieurs tentatives évaluées par score de qualité** |

### Cas d'usage idéaux :
- Classifications **ambiguës** nécessitant plusieurs analyses
- Situations où la **qualité est critique** et on peut se permettre plus de calculs
- Tâches avec des **critères de qualité mesurables** (via fonction de récompense)


In [ ]:
class RefinedTicketClassifier(dspy.Module):
    """
    Classifier avec raffinement itératif.
    
    Utilise dspy.Refine pour exécuter le classifier plusieurs fois
    et retourner la meilleure prédiction selon une fonction de récompense.
    """
    
    def __init__(self, N=3, threshold=1.0):
        super().__init__()
        self.valid_categories = {'hardware', 'software', 'network', 'account', 'application'}
        self.valid_priorities = {'critical', 'urgent', 'high', 'medium', 'low'}
        
        # Module de base à raffiner
        base_module = dspy.ChainOfThought(TicketClassifier)
        
        # Créer le module Refine avec fonction de récompense
        self.refine = dspy.Refine(
            module=base_module,
            N=N,  # Nombre de tentatives
            reward_fn=self._reward_function,  # Fonction d'évaluation
            threshold=threshold  # Arrêt anticipé si atteint
        )
    
    def _reward_function(self, args, prediction):
        """
        Fonction de récompense : évalue la qualité de la prédiction.
        
        Retourne un score entre 0.0 et 1.0 :
        - 1.0 : catégorie ET priorité valides
        - 0.5 : seulement l'un des deux valide
        - 0.0 : les deux invalides
        """
        score = 0.0
        
        # Vérifier la validité de la catégorie
        if hasattr(prediction, 'category'):
            if prediction.category.strip().lower() in self.valid_categories:
                score += 0.5
        
        # Vérifier la validité de la priorité
        if hasattr(prediction, 'priority'):
            if prediction.priority.strip().lower() in self.valid_priorities:
                score += 0.5
        
        return score
    
    def forward(self, ticket):
        # Refine exécutera le module N fois et retournera la meilleure prédiction
        result = self.refine(ticket=ticket)
        return dspy.Prediction(
            category=result.category,
            priority=result.priority
        )

### Test du classifier avec raffinement


In [ ]:
print("🔄 Test du classifier avec raffinement itératif\n")

# Créer le classifier avec 3 tentatives
refined_classifier = RefinedTicketClassifier(N=3, threshold=1.0)

# Ticket ambigu pour tester le raffinement
ticket_ambigu = "Le système affiche des messages d'erreur étranges. Parfois ça marche, parfois non."

print(f"Ticket: {ticket_ambigu}\n")

# Classifier avec raffinement
result = refined_classifier(ticket=ticket_ambigu)

print(f"✅ Résultat après raffinement:")
print(f"   Catégorie: {result.category}")
print(f"   Priorité: {result.priority}")

print("\n💡 Le module Refine a exécuté le classifier jusqu'à 3 fois")
print("   et a retourné la prédiction avec le meilleur score de qualité.")

### 📊 Comparaison : ChainOfThought vs Refine

| Aspect | ChainOfThought | Refine |
|--------|----------------|--------|
| **Nombre de tentatives** | 1 | N (configurable) |
| **Évaluation** | Aucune | Fonction de récompense |
| **Coût (appels LLM)** | Faible (1x) | Plus élevé (Nx) |
| **Qualité** | Bonne | Potentiellement meilleure |
| **Cas d'usage** | Tâches standard | Tâches critiques/ambiguës |

### 🎯 Quand utiliser Refine ?

✅ **Utilisez Refine quand :**
- La qualité est **critique** (décisions importantes)
- Les tickets sont **ambigus** ou complexes
- Vous avez une **fonction de qualité** claire
- Le **coût supplémentaire** est acceptable

❌ **Évitez Refine si :**
- Vous avez des **contraintes de latence** strictes
- Les **coûts** doivent être minimisés
- La tâche est **simple** et ChainOfThought suffit


## 6.6 Pattern d'ensemble (combiner plusieurs prédictions)

Le **pattern d'ensemble** combine les prédictions de plusieurs modèles pour améliorer la robustesse et la précision.

### Problème

- Un seul modèle peut faire des erreurs
- Différents modèles ont différentes forces
- Vous voulez maximiser la fiabilité

### Solution

Créer un module qui :
1. Exécute la prédiction avec **plusieurs modèles**
2. **Combine** les résultats (vote majoritaire, moyenne pondérée, etc.)
3. Retourne la prédiction la plus consensuelle

### Stratégies de combinaison

- **Vote majoritaire** : La prédiction la plus fréquente
- **Vote pondéré** : Chaque modèle a un poids différent
- **Consensus strict** : Tous les modèles doivent être d'accord

### Exemple: classifier d'ensemble

In [ ]:
from collections import Counter

class EnsembleTicketClassifier(dspy.Module):
    """
    Classifier d'ensemble utilisant plusieurs modèles et vote majoritaire
    """
    
    def __init__(self, models):
        """
        Args:
            models: Liste de tuples (lm, weight) où weight est le poids du modèle
        """
        super().__init__()
        self.models = models
        self.signature = TicketClassifier
    
    def forward(self, ticket):
        predictions = []
        
        # Obtenir les prédictions de chaque modèle
        for lm, weight in self.models:
            try:
                with dspy.settings.context(lm=lm):
                    predictor = dspy.ChainOfThought(self.signature)
                    result = predictor(ticket=ticket)
                    
                    # Ajouter la prédiction avec son poids
                    for _ in range(weight):
                        predictions.append({
                            'category': result.category.strip().lower(),
                            'priority': result.priority.strip().lower()
                        })
            
            except Exception as e:
                print(f"⚠️ Erreur avec un modèle: {e}")
                continue
        
        if not predictions:
            raise Exception("Aucun modèle n'a pu faire de prédiction")
        
        # Vote majoritaire pour la catégorie
        categories = [p['category'] for p in predictions]
        category_counts = Counter(categories)
        winning_category = category_counts.most_common(1)[0][0]
        
        # Vote majoritaire pour la priorité
        priorities = [p['priority'] for p in predictions]
        priority_counts = Counter(priorities)
        winning_priority = priority_counts.most_common(1)[0][0]
        
        # Normaliser les résultats
        category = next((c for c in CATEGORIES if c.lower() == winning_category), winning_category)
        priority = next((p for p in PRIORITIES if p.lower() == winning_priority), winning_priority)
        
        # Calculer la confiance (pourcentage d'accord)
        category_confidence = category_counts[winning_category] / len(predictions)
        priority_confidence = priority_counts[winning_priority] / len(predictions)
        
        return dspy.Prediction(
            category=category,
            priority=priority,
            category_confidence=category_confidence,
            priority_confidence=priority_confidence,
            num_models=len(self.models)
        )

print("✅ Classe EnsembleTicketClassifier définie")

### Test du classifier d'ensemble

In [ ]:
print("🎯 Test du classifier d'ensemble")
print("   Combinaison de 3 modèles avec vote majoritaire\n")

# Créer l'ensemble avec 3 modèles
# Format: (modèle, poids)
ensemble_classifier = EnsembleTicketClassifier([
    (lm_ollama_llama, 2),    # Poids 2 (meilleur modèle)
    (lm_ollama_qwen, 2),     # Poids 2 (aussi très bon)
    (lm_ollama_mistral, 1)   # Poids 1 (plus rapide mais moins précis)
])

# Tester sur quelques exemples
test_tickets = [
    "Mon ordinateur portable ne démarre plus, présentation urgente dans 1h",
    "Besoin d'accès au VPN pour le télétravail",
]

for i, ticket in enumerate(test_tickets, 1):
    print(f"{i}. Ticket: {ticket}")
    result = ensemble_classifier(ticket=ticket)
    print(f"   → Catégorie: {result.category} (confiance: {result.category_confidence:.1%})")
    print(f"   → Priorité: {result.priority} (confiance: {result.priority_confidence:.1%})")
    print()

# Évaluer sur l'ensemble de validation
print("📊 Évaluation de l'ensemble sur la validation:")
score_ensemble = evaluate_module(ensemble_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_ensemble:.2%}")
print()

print("💡 L'ensemble combine les forces de plusieurs modèles")
print("   pour améliorer la robustesse et la précision.")

## 6.7 Combiner les patterns

En production, vous pouvez **combiner plusieurs patterns** pour maximiser la robustesse :

### Exemple: classifier de production complet

```python
class ProductionTicketClassifier(dspy.Module):
    """
    Classifier de production avec tous les patterns
    """
    
    def __init__(self, primary_lm, fallback_lm, ensemble_models):
        super().__init__()
        self.primary_lm = primary_lm
        self.fallback_lm = fallback_lm
        self.ensemble_models = ensemble_models
        self.valid_categories = set(cat.lower() for cat in CATEGORIES)
        self.valid_priorities = set(pri.lower() for pri in PRIORITIES)
    
    def forward(self, ticket, use_ensemble=False):
        # 1. Décider de la stratégie
        if use_ensemble:
            # Utiliser l'ensemble pour les cas critiques
            classifier = EnsembleTicketClassifier(self.ensemble_models)
        else:
            # Utiliser le fallback pour les cas normaux
            classifier = FallbackTicketClassifier(self.primary_lm, self.fallback_lm)
        
        # 2. Exécuter avec retry
        max_retries = 3
        for attempt in range(max_retries):
            try:
                result = classifier(ticket=ticket)
                
                # 3. Valider et corriger
                category, priority, is_valid = self.validate_and_correct(
                    result.category,
                    result.priority
                )
                
                return dspy.Prediction(
                    category=category,
                    priority=priority,
                    is_valid=is_valid,
                    strategy='ensemble' if use_ensemble else 'fallback',
                    attempts=attempt + 1
                )
                
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)
```

### Quand utiliser chaque pattern

| Pattern | Quand l'utiliser | Coût |
|---------|-----------------|------|
| **Validation** | Toujours (critique) | Faible |
| **Retry** | APIs externes | Faible |
| **Fallback** | Haute disponibilité requise | Moyen |
| **Ensemble** | Précision maximale requise | Élevé |
| **Combinaison** | Production critique | Très élevé |

## 6.8 Résumé de la Partie 6

### Ce que nous avons appris

1. **Pattern de validation** :
   - Vérifier que les sorties respectent les contraintes
   - Corriger automatiquement les valeurs invalides
   - Normaliser les formats

2. **Pattern de retry** :
   - Réessayer automatiquement en cas d'erreur temporaire
   - Backoff exponentiel (1s, 2s, 4s, 8s...)
   - Nombre maximum de tentatives

3. **Pattern de fallback** :
   - Modèle de secours si le principal échoue
   - Garantir la disponibilité du service
   - Compromis qualité/disponibilité

4. **Pattern d'ensemble** :
   - Combiner plusieurs modèles (vote majoritaire)
   - Améliorer la robustesse
   - Mesurer la confiance

5. **Combinaison de patterns** :
   - Utiliser plusieurs patterns ensemble
   - Architecture de production robuste
   - Adapter selon les besoins

### Points clés à retenir

- ✅ **Validation** : Toujours valider les sorties en production
- ✅ **Retry** : Gérer les erreurs temporaires automatiquement
- ✅ **Fallback** : Avoir un plan B pour la disponibilité
- ✅ **Ensemble** : Utiliser pour les cas critiques nécessitant haute précision
- ✅ **Combinaison** : Adapter les patterns selon vos contraintes

### Compromis à considérer

| Aspect | Simple | Production robuste |
|--------|--------|-------------------|
| **Complexité du code** | Faible | Élevée |
| **Coût d'exécution** | Faible | Élevé (multiple LLM calls) |
| **Latence** | Rapide | Plus lente |
| **Fiabilité** | Moyenne | Très élevée |
| **Maintenance** | Simple | Plus complexe |

### Recommandations

**Pour débuter** :
- Commencer simple (sans patterns)
- Ajouter la validation dès que possible
- Ajouter les autres patterns selon les besoins

**Pour la production** :
- Validation : **obligatoire**
- Retry : **fortement recommandé** pour les APIs
- Fallback : selon l'importance de la disponibilité
- Ensemble : uniquement si la précision est critique

**Stratégie progressive** :
1. Phase 1 : Module simple + validation
2. Phase 2 : Ajouter retry pour gérer les erreurs
3. Phase 3 : Ajouter fallback pour la disponibilité
4. Phase 4 : Ensemble pour les cas critiques seulement

### Prochaines étapes

- **Partie 7** : GEPA en pratique (optimisation sophistiquée avec algorithmes génétiques)
- **Partie 8** : Conclusion et mise en production

# Partie 7: GEPA en pratique

## 7.1 Introduction à GEPA

**GEPA** (Genetic-Pareto Algorithm) est l'optimiseur le plus sophistiqué de DSPy. Il combine plusieurs techniques avancées pour améliorer automatiquement vos prompts.

### Qu'est-ce que GEPA?

GEPA utilise une approche inspirée de l'évolution biologique :

1. **🧬 Algorithmes génétiques** : Génère des "populations" de prompts qui évoluent
2. **🤔 Réflexion LLM** : Utilise un LLM pour analyser les erreurs et proposer des améliorations
3. **📊 Optimisation Pareto** : Équilibre plusieurs objectifs (précision, concision, etc.)
4. **🔄 Itérations adaptatives** : Apprend de ses erreurs pour s'améliorer

### Comment ça fonctionne?

```
┌─────────────────────────────────────────────────────────┐
│ 1. POPULATION INITIALE                                  │
│    Génère plusieurs variantes de prompts               │
│    ↓                                                    │
│ 2. ÉVALUATION                                          │
│    Teste chaque variante sur les données d'entraînement│
│    ↓                                                    │
│ 3. SÉLECTION                                           │
│    Garde les meilleurs (front de Pareto)               │
│    ↓                                                    │
│ 4. RÉFLEXION                                           │
│    LLM analyse les erreurs et propose des améliorations│
│    ↓                                                    │
│ 5. MUTATION                                            │
│    Génère de nouvelles variantes basées sur la réflexion│
│    ↓                                                    │
│ 6. RÉPÈTE jusqu'à convergence                          │
└─────────────────────────────────────────────────────────┘
```

### Pourquoi GEPA est différent?

| Optimiseur | Approche | Réflexion LLM | Amélioration typique |
|------------|----------|---------------|---------------------|
| BootstrapFewShot | Exemples fixes | ❌ Non | 5-15% |
| MIPRO | Variations systématiques | ❌ Non | 10-25% |
| **GEPA** | **Évolution + Réflexion** | **✅ Oui** | **15-30%** |

### Nouveautés dans DSPy 3.0+

GEPA a été intégré directement dans DSPy et nécessite maintenant :
- **reflection_lm** : Un modèle LLM dédié à l'analyse des erreurs
- **auto** : Niveau d'optimisation ('light', 'medium', 'heavy')
- **Métrique compatible** : Doit accepter les paramètres GEPA

## 7.2 Configuration de GEPA

### Prérequis

Avant d'utiliser GEPA, vous devez avoir :

1. ✅ **Données d'entraînement** : Au moins 15-20 exemples de qualité
2. ✅ **Données de validation** : 5-10 exemples séparés pour éviter le surapprentissage
3. ✅ **Métrique d'évaluation** : Fonction retournant un score entre 0 et 1
4. ✅ **Module à optimiser** : Votre module DSPy de base
5. ✅ **Temps** : 10-30 minutes selon le niveau d'optimisation

### Configuration du modèle de réflexion

Le **reflection_lm** est un LLM utilisé par GEPA pour analyser les erreurs et proposer des améliorations. Il est distinct du modèle principal.

**Recommandations** :
- Utiliser un modèle avec bonne capacité de raisonnement
- Température élevée (1.0) pour plus de créativité
- Max tokens élevé (8000+) pour des analyses détaillées

In [ ]:
print("🔧 Configuration de GEPA\n")

# 1. Configurer le modèle principal (celui qu'on optimise)
lm_main = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

dspy.configure(lm=lm_main)

# 2. Configurer le modèle de réflexion pour GEPA
# Important: Température élevée pour plus de créativité dans l'analyse
reflection_lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=1.0,      # Haute température = plus de créativité
    max_tokens=8000       # Tokens élevés = analyses détaillées
)

print("✅ Modèle principal configuré: llama3.1:8b (temp=0.3)")
print("✅ Modèle de réflexion configuré: llama3.1:8b (temp=1.0)")
print()

# 3. Préparer les données au format DSPy
train_examples = [
    dspy.Example(
        ticket=ex['ticket'],
        category=ex['category'],
        priority=ex['priority']
    ).with_inputs('ticket')
    for ex in trainset
]

val_examples = [
    dspy.Example(
        ticket=ex['ticket'],
        category=ex['category'],
        priority=ex['priority']
    ).with_inputs('ticket')
    for ex in valset
]

print(f"✅ Données préparées:")
print(f"   - Entraînement: {len(train_examples)} exemples")
print(f"   - Validation: {len(val_examples)} exemples")

## 7.3 Optimisation avec GEPA (mode 'light')

Le mode **'light'** est le niveau d'optimisation le plus rapide de GEPA. Il est idéal pour :
- Première expérimentation avec GEPA
- Tests rapides (5-10 minutes)
- Validation du concept
- Ressources limitées

### Niveaux d'optimisation GEPA

| Niveau | Temps estimé | Appels LLM | Amélioration | Usage |
|--------|-------------|------------|--------------|-------|
| **light** | 5-10 min | ~200-400 | 10-20% | Tests, prototypage |
| **medium** | 10-20 min | ~400-800 | 15-25% | Production légère |
| **heavy** | 20-40 min | ~800-1600 | 20-30% | Maximum performance |

### Exemple pratique

In [ ]:
from dspy.teleprompt import GEPA

print("=" * 70)
print("🧬 OPTIMISATION GEPA - MODE 'LIGHT'")
print("=" * 70)
print()

# 1. Créer le module à optimiser
print("1️⃣ Création du module de base...")
baseline_classifier = SimpleTicketClassifier()

# 2. Évaluer AVANT optimisation
print("2️⃣ Évaluation AVANT optimisation...")
score_before = evaluate_module(baseline_classifier, val_examples, exact_match_metric)
print(f"   📊 Score baseline: {score_before:.2%}\n")

# 3. Configurer l'optimiseur GEPA
print("3️⃣ Configuration de l'optimiseur GEPA...")
optimizer = GEPA(
    metric=exact_match_metric,
    auto='light',                    # Mode rapide
    reflection_lm=reflection_lm      # Modèle pour l'analyse des erreurs
)
print("   ✅ Optimiseur configuré (mode: light)\n")

# 4. Lancer l'optimisation
print("4️⃣ Lancement de l'optimisation GEPA...")
print("   ⏰ Cela va prendre 5-10 minutes avec Ollama")
print("   ☕ C'est le moment de prendre un café!\n")

try:
    optimized_classifier = optimizer.compile(
        student=baseline_classifier,
        trainset=train_examples,
        valset=val_examples
    )
    
    print("\n✅ Optimisation GEPA terminée!\n")
    
    # 5. Évaluer APRÈS optimisation
    print("5️⃣ Évaluation APRÈS optimisation...")
    score_after = evaluate_module(optimized_classifier, val_examples, exact_match_metric)
    print(f"   📊 Score optimisé: {score_after:.2%}\n")
    
    # 6. Calculer l'amélioration
    improvement = ((score_after - score_before) / score_before) * 100 if score_before > 0 else 0
    improvement_abs = score_after - score_before
    
    print("=" * 70)
    print("📈 RÉSULTATS DE L'OPTIMISATION")
    print("=" * 70)
    print(f"Score AVANT:      {score_before:.2%}")
    print(f"Score APRÈS:      {score_after:.2%}")
    print(f"Amélioration:     {improvement_abs:+.2%} ({improvement:+.1f}%)")
    print("=" * 70)
    
except Exception as e:
    print(f"\n❌ Erreur lors de l'optimisation GEPA: {e}")
    print("   Vérifiez que:")
    print("   - Ollama est en cours d'exécution")
    print("   - Le modèle llama3.1:8b est téléchargé")
    print("   - Vous avez suffisamment de mémoire disponible")

## 7.4 Analyser les prompts optimisés

Une des forces de GEPA est qu'il **génère des prompts explicites** que vous pouvez inspecter et comprendre. Cela permet de :
- 🔍 Voir ce que GEPA a changé
- 📚 Apprendre comment améliorer vos prompts manuellement
- ✅ Valider que les modifications ont du sens
- 🎓 Comprendre pourquoi la performance s'est améliorée

### Inspection des prompts

In [ ]:
print("🔍 Inspection des prompts optimisés par GEPA\n")

# Essayer d'accéder aux prompts optimisés
if hasattr(optimized_classifier, 'classifier'):
    predictor = optimized_classifier.classifier
    
    # 1. Signature optimisée
    if hasattr(predictor, 'extended_signature'):
        print("=" * 70)
        print("📝 SIGNATURE OPTIMISÉE")
        print("=" * 70)
        sig = predictor.extended_signature
        print(f"Docstring: {sig.__doc__}")
        print()
        
        # Afficher les champs
        if hasattr(sig, 'input_fields'):
            print("Champs d'entrée:")
            for name, field in sig.input_fields.items():
                desc = getattr(field, 'desc', 'N/A')
                print(f"  - {name}: {desc}")
        
        if hasattr(sig, 'output_fields'):
            print("\nChamps de sortie:")
            for name, field in sig.output_fields.items():
                desc = getattr(field, 'desc', 'N/A')
                print(f"  - {name}: {desc}")
        print()
    
    # 2. Exemples de démonstration
    if hasattr(predictor, 'demos') and predictor.demos:
        print("=" * 70)
        print("📚 EXEMPLES DE DÉMONSTRATION GÉNÉRÉS")
        print("=" * 70)
        print(f"Nombre d'exemples: {len(predictor.demos)}\n")
        
        # Afficher les 3 premiers exemples
        for i, demo in enumerate(predictor.demos[:3], 1):
            print(f"Exemple {i}:")
            print(f"  Ticket: {demo.ticket[:80]}...")
            if hasattr(demo, 'category'):
                print(f"  Catégorie: {demo.category}")
            if hasattr(demo, 'priority'):
                print(f"  Priorité: {demo.priority}")
            print()
    
    else:
        print("ℹ️ Aucun exemple de démonstration trouvé")
        print("   (GEPA peut optimiser uniquement les instructions)")
    
else:
    print("ℹ️ Structure du module différente")
    print("   L'optimisation s'est concentrée sur d'autres aspects")

print("=" * 70)
print("💡 Ce que GEPA a fait:")
print("=" * 70)
print("✅ Analysé les erreurs sur les données d'entraînement")
print("✅ Généré des variantes de prompts")
print("✅ Utilisé la réflexion LLM pour proposer des améliorations")
print("✅ Sélectionné la meilleure configuration via Pareto")
print("=" * 70)

## 7.5 Tester le classifier optimisé

Maintenant que GEPA a optimisé notre classifier, testons-le sur de nouveaux exemples pour voir la différence.

### Comparaison côte à côte

In [ ]:
print("🧪 Test comparatif: Baseline vs GEPA optimisé\n")

# Exemples de test
test_cases = [
    {
        'ticket': "Mon ordinateur portable ne s'allume plus, j'ai une réunion importante dans 1h",
        'expected_category': 'Hardware',
        'expected_priority': 'Urgent'
    },
    {
        'ticket': "Je voudrais accès à la base de données pour faire des analyses",
        'expected_category': 'Account',
        'expected_priority': 'Medium'
    },
    {
        'ticket': "Le WiFi est complètement HS dans tout le bâtiment!",
        'expected_category': 'Network',
        'expected_priority': 'Critical'
    },
    {
        'ticket': "Mon logiciel de comptabilité plante quand j'exporte en PDF",
        'expected_category': 'Software',
        'expected_priority': 'High'
    }
]

print("=" * 100)
print(f"{'Ticket':<50} | {'Attendu':<20} | {'Baseline':<20} | {'GEPA':<20}")
print("=" * 100)

correct_baseline = 0
correct_gepa = 0

for test in test_cases:
    ticket = test['ticket']
    expected = f"{test['expected_category']}/{test['expected_priority']}"
    
    # Prédiction baseline
    pred_baseline = baseline_classifier(ticket=ticket)
    baseline_result = f"{pred_baseline.category}/{pred_baseline.priority}"
    baseline_match = (pred_baseline.category == test['expected_category'] and 
                     pred_baseline.priority == test['expected_priority'])
    
    # Prédiction GEPA
    pred_gepa = optimized_classifier(ticket=ticket)
    gepa_result = f"{pred_gepa.category}/{pred_gepa.priority}"
    gepa_match = (pred_gepa.category == test['expected_category'] and 
                 pred_gepa.priority == test['expected_priority'])
    
    if baseline_match:
        correct_baseline += 1
    if gepa_match:
        correct_gepa += 1
    
    # Afficher avec indicateurs de succès
    baseline_icon = "✅" if baseline_match else "❌"
    gepa_icon = "✅" if gepa_match else "❌"
    
    print(f"{ticket[:48]:<50} | {expected:<20} | {baseline_icon} {baseline_result:<17} | {gepa_icon} {gepa_result:<17}")

print("=" * 100)
print(f"Précision sur les exemples de test:")
print(f"  Baseline: {correct_baseline}/{len(test_cases)} ({correct_baseline/len(test_cases)*100:.0f}%)")
print(f"  GEPA:     {correct_gepa}/{len(test_cases)} ({correct_gepa/len(test_cases)*100:.0f}%)")
print("=" * 100)

## 7.6 Conseils pour optimiser avec GEPA

### 7.6.1 Qualité des données

La qualité de l'optimisation GEPA dépend **fortement** de vos données :

✅ **Bonnes pratiques** :
- Minimum 15-20 exemples d'entraînement (idéalement 30-50)
- Exemples **diversifiés** couvrant tous les cas d'usage
- Labels **corrects** et **cohérents**
- Données de validation **séparées** du trainset

❌ **À éviter** :
- Trop peu d'exemples (<10)
- Exemples répétitifs ou très similaires
- Labels incohérents ou ambigus
- Utiliser les mêmes données pour train et validation

### 7.6.2 Choix de la métrique

Votre **métrique détermine ce que GEPA optimise** :

```python
# Métrique stricte (tout ou rien)
def exact_match(example, prediction, trace=None, pred_name=None, pred_trace=None):
    return 1.0 if (prediction.category == example.category and 
                   prediction.priority == example.priority) else 0.0

# Métrique avec crédit partiel (souvent meilleure pour GEPA)
def partial_match(example, prediction, trace=None, pred_name=None, pred_trace=None):
    category_match = prediction.category == example.category
    priority_match = prediction.priority == example.priority
    
    if category_match and priority_match:
        return 1.0
    elif category_match:
        return 0.7  # Catégorie plus importante
    elif priority_match:
        return 0.5
    else:
        return 0.0
```

💡 **Conseil** : Les métriques avec crédit partiel donnent généralement de meilleurs résultats car elles fournissent plus de signal d'apprentissage à GEPA.

### 7.6.3 Choix du niveau d'optimisation

| Situation | Niveau recommandé | Raison |
|-----------|------------------|---------|
| Première expérimentation | **light** | Test rapide du concept |
| Prototype pour démo | **light** | Balance vitesse/performance |
| Application production (non-critique) | **medium** | Bon compromis |
| Application production (critique) | **heavy** | Maximum de performance |
| Recherche / benchmark | **heavy** | Explorer les limites |

### 7.6.4 Configuration du modèle de réflexion

Le **reflection_lm** influence la qualité des améliorations :

✅ **Bonnes pratiques** :
- Utiliser un modèle avec bonne capacité de raisonnement
- Température élevée (0.8-1.2) pour la créativité
- Max tokens élevé (6000-10000) pour analyses détaillées
- Peut être le même modèle que le modèle principal

❌ **À éviter** :
- Modèles trop petits (<7B paramètres)
- Température trop basse (<0.5)
- Max tokens trop faible (<4000)

### 7.6.5 Éviter le surapprentissage

GEPA peut **surapprendre** sur les données d'entraînement :

✅ **Prévention** :
- Toujours avoir un **valset séparé**
- Valider sur de **nouvelles données** après optimisation
- Comparer les scores train vs validation
- Si grand écart : vos données ne sont pas assez diversifiées

```python
# Bon: Évaluation sur données séparées
score_train = evaluate_module(optimized, train_examples, metric)
score_val = evaluate_module(optimized, val_examples, metric)

if score_train - score_val > 0.2:
    print("⚠️ Possible surapprentissage!")
```

## 7.7 Troubleshooting et erreurs courantes

### Erreur 1: `TypeError: GEPA.__init__() got an unexpected keyword argument`

**Symptôme** :
```
TypeError: GEPA.__init__() got an unexpected keyword argument 'breadth'
```

**Cause** : Utilisation de paramètres de l'ancienne API GEPA (pre-3.0)

**Solution** :
```python
# ❌ Ancienne API (ne fonctionne plus)
optimizer = GEPA(
    metric=my_metric,
    breadth=10,
    depth=3
)

# ✅ Nouvelle API (DSPy 3.0+)
optimizer = GEPA(
    metric=my_metric,
    auto='light',  # ou 'medium', 'heavy'
    reflection_lm=reflection_lm
)
```

### Erreur 2: `TypeError: metric() missing required positional argument`

**Symptôme** :
```
TypeError: metric() missing 2 required positional arguments: 'pred_name' and 'pred_trace'
```

**Cause** : Métrique pas compatible avec l'API GEPA

**Solution** : Ajouter les paramètres optionnels à votre métrique
```python
# ❌ Ancienne signature
def my_metric(example, prediction):
    return 1.0 if prediction.category == example.category else 0.0

# ✅ Nouvelle signature (compatible GEPA)
def my_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    return 1.0 if prediction.category == example.category else 0.0
```

### Erreur 3: Ollama timeout ou erreur de connexion

**Symptôme** :
```
ConnectionError: Failed to connect to Ollama
```

**Causes possibles** :
1. Ollama n'est pas démarré
2. Modèle non téléchargé
3. Mémoire insuffisante

**Solutions** :
```bash
# 1. Vérifier qu'Ollama tourne
ollama list

# 2. Démarrer Ollama si nécessaire
ollama serve

# 3. Télécharger le modèle
ollama pull llama3.1:8b

# 4. Vérifier la mémoire disponible
# GEPA + Ollama nécessite ~8-12 GB RAM
```

### Erreur 4: GEPA ne s'améliore pas

**Symptôme** : Score après optimisation ≈ score avant

**Causes possibles** :
1. Données d'entraînement insuffisantes ou de mauvaise qualité
2. Métrique mal définie
3. Tâche trop difficile pour le modèle
4. Module déjà bien optimisé

**Solutions** :
```python
# Vérifier la qualité des données
print(f"Nombre d'exemples train: {len(trainset)}")
print(f"Nombre d'exemples val: {len(valset)}")

# Vérifier la métrique
for ex in trainset[:5]:
    pred = baseline(ticket=ex['ticket'])
    score = metric(ex, pred)
    print(f"Score: {score} | Pred: {pred.category}/{pred.priority} | Truth: {ex['category']}/{ex['priority']}")

# Essayer un niveau plus élevé
optimizer = GEPA(
    metric=metric,
    auto='heavy',  # au lieu de 'light'
    reflection_lm=reflection_lm
)
```

### Erreur 5: Out of Memory (OOM)

**Symptôme** : Ollama ou Python crash avec erreur de mémoire

**Solutions** :
1. Utiliser un modèle plus petit : `mistral:7b` au lieu de `llama3.1:8b`
2. Réduire `max_tokens` du reflection_lm
3. Utiliser `auto='light'` au lieu de `medium` ou `heavy`
4. Fermer les autres applications
5. Utiliser une API cloud au lieu d'Ollama local

## 7.8 Résumé de la Partie 7

### Ce que nous avons appris

1. **GEPA : L'optimiseur le plus sophistiqué**
   - Combine algorithmes génétiques + réflexion LLM + Pareto
   - Amélioration typique : 15-30%
   - Nécessite un modèle de réflexion (reflection_lm)

2. **Configuration essentielle**
   - `auto` : 'light', 'medium', ou 'heavy'
   - `reflection_lm` : Modèle pour analyser les erreurs (temp=1.0, max_tokens=8000)
   - `metric` : Doit accepter les paramètres GEPA (trace, pred_name, pred_trace)

3. **Niveaux d'optimisation**
   - **light** : 5-10 min, ~200-400 appels LLM, 10-20% amélioration
   - **medium** : 10-20 min, ~400-800 appels LLM, 15-25% amélioration
   - **heavy** : 20-40 min, ~800-1600 appels LLM, 20-30% amélioration

4. **Inspection des résultats**
   - Voir les prompts optimisés
   - Comprendre les changements
   - Valider les améliorations

5. **Bonnes pratiques**
   - Données : 15-20+ exemples diversifiés
   - Métrique : Crédit partiel souvent meilleur
   - Validation : Toujours sur données séparées
   - Modèle : ≥7B paramètres recommandé

6. **Troubleshooting**
   - Erreurs d'API : Vérifier la signature de la métrique
   - Pas d'amélioration : Vérifier qualité des données
   - OOM : Réduire niveau ou utiliser modèle plus petit

### Points clés à retenir

- ✅ **GEPA est puissant** mais nécessite temps et ressources
- ✅ **Commencer avec 'light'** pour tester le concept
- ✅ **Qualité des données = qualité des résultats**
- ✅ **Toujours valider** sur données séparées
- ✅ **Inspecter les prompts** pour comprendre les améliorations

### Quand utiliser GEPA?

| Situation | GEPA? | Alternative |
|-----------|-------|-------------|
| Prototypage rapide | ❌ Non | Module simple |
| Tests initiaux | ❌ Non | BootstrapFewShot |
| Application production (non-critique) | ⚠️ Peut-être | MIPRO |
| Application production (critique) | ✅ Oui | GEPA medium/heavy |
| Recherche / Maximum performance | ✅ Oui | GEPA heavy |
| Ressources limitées | ❌ Non | BootstrapFewShot |

### Workflow recommandé

```python
# Phase 1: Baseline
classifier = SimpleTicketClassifier()
score_baseline = evaluate(classifier, valset, metric)

# Phase 2: Optimisation simple
from dspy.teleprompt import BootstrapFewShot
optimizer_simple = BootstrapFewShot(metric=metric)
classifier_simple = optimizer_simple.compile(classifier, trainset)
score_simple = evaluate(classifier_simple, valset, metric)

# Phase 3: GEPA (si amélioration justifie le temps)
if score_simple < target_score:
    from dspy.teleprompt import GEPA
    optimizer_gepa = GEPA(metric=metric, auto='light', reflection_lm=reflection_lm)
    classifier_gepa = optimizer_gepa.compile(classifier, trainset, valset)
    score_gepa = evaluate(classifier_gepa, valset, metric)
    
    # Phase 4: GEPA heavy si nécessaire
    if score_gepa < target_score:
        optimizer_heavy = GEPA(metric=metric, auto='heavy', reflection_lm=reflection_lm)
        classifier_final = optimizer_heavy.compile(classifier, trainset, valset)
```

### Ressources additionnelles

- 📄 [Paper GEPA (arXiv)](https://arxiv.org/abs/2507.19457)
- 💻 [GitHub GEPA](https://github.com/gepa-ai/gepa)
- 📖 [Documentation DSPy](https://dspy-docs.vercel.app/)

### Prochaines étapes

- **Partie 8** : Conclusion et mise en production

# Partie 8: Conclusion et mise en production

## 8.1 Récapitulatif du parcours

Félicitations! 🎉 Vous avez parcouru un tutoriel complet sur DSPy et GEPA. Récapitulons ce que vous avez appris :

### Partie 0: Configuration
✅ Installation de DSPy et Ollama  
✅ Configuration des modèles locaux  
✅ Chargement des données d'entraînement

### Partie 1: Signatures
✅ Définition des entrées/sorties avec `dspy.Signature`  
✅ 5 exemples progressifs de signatures  
✅ Bonnes pratiques pour des signatures efficaces

### Partie 2: Modules
✅ `Predict` : Module de base  
✅ `ChainOfThought` : Raisonnement explicite  
✅ `ReAct` : Raisonnement + actions  
✅ `ProgramOfThought` : Code + raisonnement  
✅ Composition de modules (séquentiel, validation, ensemble)

### Partie 3: Évaluation
✅ Métriques d'évaluation (exact match, partial match)  
✅ Fonction `evaluate_module`  
✅ Comparaison de différents modules

### Partie 4: Optimiseurs
✅ `BootstrapFewShot` : Génération d'exemples  
✅ `MIPRO` : Optimisation instructions + exemples  
✅ `SignatureOptimizer` : Optimisation d'instructions  
✅ Comparaison et guide de sélection

### Partie 5: Multi-modèles
✅ Configuration Ollama, OpenAI, Anthropic  
✅ Benchmarking de modèles  
✅ Architectures hybrides  
✅ Guide de sélection par cas d'usage

### Partie 6: Patterns avancés
✅ Validation des sorties  
✅ Retry automatique  
✅ Fallback (modèle de secours)  
✅ Ensemble (vote majoritaire)  
✅ Combinaison de patterns

### Partie 7: GEPA en pratique
✅ Configuration et utilisation de GEPA  
✅ Niveaux d'optimisation (light/medium/heavy)  
✅ Inspection des prompts optimisés  
✅ Bonnes pratiques et troubleshooting

**Vous maîtrisez maintenant les concepts fondamentaux et avancés de DSPy!** 🚀

## 8.2 Checklist de mise en production

Avant de déployer votre application DSPy en production, voici une checklist complète :

### 8.2.1 Données et métriques

- [ ] **Données d'entraînement de qualité**
  - Au moins 30-50 exemples diversifiés
  - Labels vérifiés et cohérents
  - Couverture de tous les cas d'usage importants
  
- [ ] **Données de validation séparées**
  - 15-20% des données totales
  - Jamais utilisées pour l'entraînement
  - Représentatives de la production
  
- [ ] **Métrique bien définie**
  - Reflète les objectifs métier
  - Compatible avec GEPA (paramètres optionnels)
  - Testée sur des cas limites

- [ ] **Données de test pour la production**
  - Ensemble de test complètement séparé
  - Mis à jour régulièrement
  - Utilisé pour monitoring continu

### 8.2.2 Optimisation

- [ ] **Module de base testé**
  - Fonctionne correctement sur les cas simples
  - Performance baseline documentée
  - Code propre et maintenable

- [ ] **Optimisation appropriée**
  - BootstrapFewShot ou MIPRO pour démarrer
  - GEPA si performance critique
  - Niveau d'optimisation adapté aux contraintes

- [ ] **Validation croisée**
  - Performance train vs validation vérifiée
  - Pas de surapprentissage détecté
  - Tests sur données réelles de production

### 8.2.3 Robustesse

- [ ] **Validation des sorties**
  - Vérification des formats
  - Valeurs dans les plages attendues
  - Gestion des cas invalides

- [ ] **Gestion des erreurs**
  - Retry pour les erreurs temporaires
  - Fallback si modèle principal échoue
  - Logs détaillés pour le debugging

- [ ] **Monitoring**
  - Métriques de performance en temps réel
  - Alertes sur dégradation
  - Collecte des erreurs

### 8.2.4 Infrastructure

- [ ] **Choix du modèle**
  - Ollama pour confidentialité/coûts
  - API cloud pour disponibilité/performance
  - Architecture hybride pour optimiser

- [ ] **Ressources suffisantes**
  - RAM : 8-16 GB pour Ollama
  - CPU/GPU : Selon le modèle
  - Bande passante : Pour APIs cloud

- [ ] **Sécurité**
  - Clés API stockées dans variables d'environnement
  - Pas de secrets dans le code
  - Logs ne contenant pas de données sensibles

### 8.2.5 Documentation

- [ ] **Code documenté**
  - Docstrings pour toutes les classes/fonctions
  - Commentaires pour la logique complexe
  - README avec instructions de déploiement

- [ ] **Tests automatisés**
  - Tests unitaires pour les composants
  - Tests d'intégration end-to-end
  - Tests de régression sur données fixes

- [ ] **Versioning**
  - Git avec commits clairs
  - Tags pour les versions de production
  - Historique des performances

## 8.3 Adapter ce tutoriel à votre cas d'usage

Ce tutoriel utilise la classification de tickets IT comme exemple, mais DSPy peut être appliqué à de nombreux cas d'usage. Voici comment adapter ce code à votre problème.

### 8.3.1 Définir votre tâche

**Questions à se poser** :
1. Quelle est mon entrée? (texte, image, tableau, etc.)
2. Quelle est ma sortie attendue? (classification, extraction, génération, etc.)
3. Ai-je des exemples d'entrée/sortie?
4. Comment mesurer si la sortie est correcte?

### 8.3.2 Créer votre signature

```python
# Exemple 1: Analyse de sentiments
class SentimentAnalysis(dspy.Signature):
    """Analyser le sentiment d'un avis client"""
    review = dspy.InputField(desc="Avis client en texte libre")
    sentiment = dspy.OutputField(desc="Sentiment: Positif, Négatif, ou Neutre")
    confidence = dspy.OutputField(desc="Niveau de confiance: Faible, Moyen, Élevé")

# Exemple 2: Extraction d'informations
class InformationExtraction(dspy.Signature):
    """Extraire des informations structurées d'un texte"""
    text = dspy.InputField(desc="Texte source")
    entities = dspy.OutputField(desc="Entités trouvées (personnes, lieux, organisations)")
    dates = dspy.OutputField(desc="Dates mentionnées")
    
# Exemple 3: Génération de contenu
class ContentGeneration(dspy.Signature):
    """Générer une description de produit marketing"""
    product_name = dspy.InputField(desc="Nom du produit")
    features = dspy.InputField(desc="Liste des caractéristiques principales")
    tone = dspy.InputField(desc="Ton souhaité: Professionnel, Décontracté, Technique")
    description = dspy.OutputField(desc="Description marketing en 2-3 phrases")

# Exemple 4: Question-réponse
class QuestionAnswering(dspy.Signature):
    """Répondre à une question basée sur un contexte"""
    context = dspy.InputField(desc="Contexte ou document source")
    question = dspy.InputField(desc="Question de l'utilisateur")
    answer = dspy.OutputField(desc="Réponse concise basée sur le contexte")
    source = dspy.OutputField(desc="Citation du contexte utilisé pour répondre")
```

### 8.3.3 Préparer vos données

```python
# Format de base pour vos données
trainset = [
    {
        'input_field1': 'valeur1',
        'input_field2': 'valeur2',
        'output_field': 'sortie attendue'
    },
    # ... plus d'exemples
]

# Convertir au format DSPy
train_examples = [
    dspy.Example(
        input_field1=ex['input_field1'],
        input_field2=ex['input_field2'],
        output_field=ex['output_field']
    ).with_inputs('input_field1', 'input_field2')
    for ex in trainset
]
```

### 8.3.4 Définir votre métrique

```python
def your_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    """
    Métrique adaptée à votre tâche
    
    Retourne un score entre 0.0 et 1.0
    """
    # Exemple 1: Exact match
    if prediction.output_field == example.output_field:
        return 1.0
    else:
        return 0.0
    
    # Exemple 2: Similarité partielle
    # from difflib import SequenceMatcher
    # similarity = SequenceMatcher(None, 
    #     prediction.output_field.lower(), 
    #     example.output_field.lower()
    # ).ratio()
    # return similarity
    
    # Exemple 3: Métrique composite
    # score = 0.0
    # if prediction.field1 == example.field1:
    #     score += 0.5
    # if prediction.field2 == example.field2:
    #     score += 0.5
    # return score
```

### 8.3.5 Template de démarrage complet

```python
import dspy

# 1. Configuration
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)
dspy.configure(lm=lm)

# 2. Signature
class YourTask(dspy.Signature):
    """Description claire de votre tâche"""
    input_field = dspy.InputField(desc="Description de l'entrée")
    output_field = dspy.OutputField(desc="Description de la sortie")

# 3. Module
class YourModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predictor = dspy.ChainOfThought(YourTask)
    
    def forward(self, input_field):
        result = self.predictor(input_field=input_field)
        return dspy.Prediction(output_field=result.output_field)

# 4. Données
trainset = [...]  # Vos données
train_examples = [
    dspy.Example(**ex).with_inputs('input_field')
    for ex in trainset
]

# 5. Métrique
def your_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    return 1.0 if prediction.output_field == example.output_field else 0.0

# 6. Évaluation
module = YourModule()
from Partie3 import evaluate_module  # Utiliser la fonction du tutoriel
score = evaluate_module(module, train_examples[:10], your_metric)
print(f"Score baseline: {score:.2%}")

# 7. Optimisation (optionnel)
from dspy.teleprompt import BootstrapFewShot
optimizer = BootstrapFewShot(metric=your_metric, max_bootstrapped_demos=3)
optimized_module = optimizer.compile(module, trainset=train_examples)
score_optimized = evaluate_module(optimized_module, train_examples[:10], your_metric)
print(f"Score optimisé: {score_optimized:.2%}")
```

## 8.4 Prochaines étapes et apprentissage continu

### 8.4.1 Approfondir DSPy

**Concepts avancés à explorer** :

1. **Retrieval-Augmented Generation (RAG)**
   - Combiner DSPy avec des bases de données vectorielles
   - Modules : `dspy.Retrieve`, `dspy.RetrieveThenGenerate`
   - Cas d'usage : QA sur documents, chatbots avec connaissance spécifique

2. **Agents multi-étapes**
   - Chaînes de décisions complexes
   - Utiliser `ReAct` pour des tâches itératives
   - Cas d'usage : Assistants autonomes, automatisation de workflows

3. **Fine-tuning de modèles**
   - DSPy peut générer des datasets pour fine-tuning
   - Exporter les prompts optimisés comme données d'entraînement
   - Cas d'usage : Modèles spécialisés pour votre domaine

4. **Optimiseurs avancés**
   - `BayesianSignatureOptimizer` : Optimisation bayésienne
   - `KNNFewShot` : Sélection dynamique d'exemples
   - Combinaison d'optimiseurs

### 8.4.2 Cas d'usage inspirants

**Applications réelles de DSPy** :

- 🏥 **Santé** : Extraction d'informations médicales, classification de symptômes
- 💼 **Entreprise** : Analyse de contrats, classification de documents
- 🎓 **Éducation** : Correction automatique, génération de quiz
- 🛒 **E-commerce** : Catégorisation de produits, recommandations
- 📰 **Médias** : Résumés d'articles, détection de fake news
- 💬 **Customer Support** : Classification de tickets (comme ce tutoriel!), routage automatique

### 8.4.3 Ressources recommandées

**Documentation officielle** :
- 📖 [DSPy Documentation](https://dspy-docs.vercel.app/) - Guide complet
- 💻 [DSPy GitHub](https://github.com/stanfordnlp/dspy) - Code source et exemples
- 📄 [Paper DSPy](https://arxiv.org/abs/2310.03714) - Recherche originale

**GEPA** :
- 📄 [Paper GEPA](https://arxiv.org/abs/2507.19457) - Algorithme détaillé
- 💻 [GEPA GitHub](https://github.com/gepa-ai/gepa) - Implémentation

**Ollama** :
- 📖 [Ollama Documentation](https://ollama.ai/docs) - Guide d'installation
- 🤖 [Ollama Models Library](https://ollama.ai/library) - Catalogue de modèles
- 💬 [Ollama Discord](https://discord.gg/ollama) - Communauté active

**Communauté et support** :
- 💬 [DSPy Discord](https://discord.gg/dspy) - Discussions et aide
- 🐦 [Twitter @DSPy_ai](https://twitter.com/dspy_ai) - Actualités
- 📺 [Tutoriels YouTube](https://www.youtube.com/results?search_query=dspy+tutorial) - Vidéos explicatives

### 8.4.4 Contribuer à l'écosystème

**Façons de contribuer** :

1. **Partager vos cas d'usage**
   - Publier vos expériences sur GitHub
   - Écrire des articles de blog
   - Présenter dans des meetups

2. **Améliorer la documentation**
   - Signaler des erreurs ou ambiguïtés
   - Proposer de nouveaux exemples
   - Traduire la documentation

3. **Développer des extensions**
   - Nouveaux optimiseurs
   - Nouveaux modules
   - Intégrations avec d'autres outils

4. **Aider la communauté**
   - Répondre aux questions sur Discord
   - Partager vos solutions
   - Mentorer les débutants

## 8.5 Conclusion

### Le pouvoir de DSPy

DSPy représente un **changement de paradigme** dans le développement avec les LLMs :

**Avant DSPy** :
- ❌ Prompts écrits manuellement et fragiles
- ❌ Difficile de maintenir la cohérence
- ❌ Optimisation par essai-erreur
- ❌ Code spécifique à chaque modèle

**Avec DSPy** :
- ✅ Prompts optimisés automatiquement
- ✅ Abstraction propre et maintenable
- ✅ Optimisation algorithmique (GEPA, MIPRO)
- ✅ Indépendance du fournisseur LLM

### Principes clés à retenir

1. **Déclaratif > Impératif**
   - Définissez *ce que* vous voulez (Signature)
   - Laissez DSPy déterminer *comment* le faire

2. **Optimisation algorithmique > Ingénierie manuelle**
   - Les optimiseurs trouvent de meilleurs prompts que l'humain
   - GEPA utilise la réflexion LLM pour s'améliorer

3. **Composition > Monolithe**
   - Construisez des systèmes complexes à partir de modules simples
   - Chaque module a une responsabilité claire

4. **Mesure > Intuition**
   - Définissez des métriques claires
   - Validez sur des données réelles
   - Itérez basé sur les données

5. **Flexibilité > Lock-in**
   - Changez de modèle sans changer le code
   - Testez facilement différentes approches
   - Adaptez selon vos contraintes

### Votre parcours commence maintenant

Vous avez maintenant tous les outils pour :
- 🚀 Construire des applications LLM robustes et performantes
- 🔧 Optimiser automatiquement vos prompts avec GEPA
- 🎯 Adapter DSPy à vos cas d'usage spécifiques
- 🌟 Contribuer à l'écosystème DSPy

### Message final

L'IA générative évolue rapidement. **DSPy vous donne une base solide** pour construire des systèmes qui :
- Évoluent avec les nouveaux modèles
- S'améliorent automatiquement
- Restent maintenables à long terme

**N'attendez plus** :
1. Identifiez un problème dans votre domaine
2. Créez une signature DSPy
3. Collectez quelques exemples
4. Laissez GEPA optimiser
5. Déployez en production

Chaque grande application commence par un premier exemple. **Le vôtre est à portée de main.**

---

### Remerciements

Merci d'avoir suivi ce tutoriel jusqu'au bout! 🙏

Si ce tutoriel vous a été utile :
- ⭐ Donnez une étoile au projet sur GitHub
- 💬 Partagez vos réussites avec la communauté
- 🐛 Signalez les erreurs ou améliorations possibles
- 🤝 Aidez d'autres développeurs à démarrer avec DSPy

**Bon développement avec DSPy et GEPA!** 🎉

---

*Ce tutoriel a été créé pour aider la communauté francophone à découvrir DSPy et GEPA. N'hésitez pas à l'adapter, le partager et le faire évoluer.*

In [ ]:
print("=" * 80)
print(" " * 25 + "🎓 TUTORIEL TERMINÉ! 🎓")
print("=" * 80)
print()
print("Vous avez complété le tutoriel DSPy et GEPA!")
print()
print("📊 Résumé de ce que vous avez appris:")
print()
print("   ✅ Partie 0: Configuration et installation")
print("   ✅ Partie 1: Signatures (5 exemples)")
print("   ✅ Partie 2: Modules (Predict, ChainOfThought, ReAct, ProgramOfThought)")
print("   ✅ Partie 3: Évaluation et métriques")
print("   ✅ Partie 4: Optimiseurs (BootstrapFewShot, MIPRO, etc.)")
print("   ✅ Partie 5: Multi-modèles et architectures hybrides")
print("   ✅ Partie 6: Patterns avancés (validation, retry, fallback, ensemble)")
print("   ✅ Partie 7: GEPA en pratique")
print("   ✅ Partie 8: Conclusion et mise en production")
print()
print("=" * 80)
print()
print("🚀 Prochaines étapes suggérées:")
print()
print("   1. Adaptez ce code à votre cas d'usage")
print("   2. Collectez vos propres données d'entraînement")
print("   3. Expérimentez avec différents modèles")
print("   4. Optimisez avec GEPA")
print("   5. Déployez en production!")
print()
print("=" * 80)
print()
print("💡 Ressources:")
print()
print("   📖 Documentation: https://dspy-docs.vercel.app/")
print("   💻 GitHub DSPy:   https://github.com/stanfordnlp/dspy")
print("   💬 Discord:       https://discord.gg/dspy")
print("   📄 Paper GEPA:    https://arxiv.org/abs/2507.19457")
print()
print("=" * 80)
print()
print("Bon développement avec DSPy! 🎉")
print()
print("=" * 80)

In [ ]:
# Just add a ‘no_think: str’ input field on your signature and pass it ‘foo.Predict(no_think=“/no_think”’

# https://github.com/Columbia-NLP-Lab/PAPILLON/blob/main/papillon_tutorial.ipynb

